<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-JSON-files" data-toc-modified-id="Load-JSON-files-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load JSON files</a></span></li><li><span><a href="#Get-annotated-entities" data-toc-modified-id="Get-annotated-entities-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Get annotated entities</a></span><ul class="toc-item"><li><span><a href="#All-docs-in-a-folder" data-toc-modified-id="All-docs-in-a-folder-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>All docs in a folder</a></span></li><li><span><a href="#Relationships" data-toc-modified-id="Relationships-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Relationships</a></span></li></ul></li><li><span><a href="#Metrics" data-toc-modified-id="Metrics-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Metrics</a></span></li><li><span><a href="#Create-Matrix-for-Potential-Relationships-from-Annotated-Articles" data-toc-modified-id="Create-Matrix-for-Potential-Relationships-from-Annotated-Articles-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Create Matrix for Potential Relationships from Annotated Articles</a></span></li></ul></div>

In [10]:
import pandas as pd
import numpy as np
import os

In [11]:
import difflib

In [12]:
import re

In [13]:
import json

In [15]:
import spacy

In [16]:
import en_core_web_lg
nlp_lg = spacy.load('en_core_web_lg')

In [17]:
from spacy.matcher import Matcher

## Load JSON files 

In [78]:
JSONfiles = [x for x in os.listdir('tagtog/BodyOnly') if x.endswith('.json')]
JSONfiles

['aBbvd8LscQsoqdLvBQHVvC6_sDjq-Archer2017_Article_EndolithicMicrobialDiversityIn_body.txt.ann.json',
 'a2Z6iG1EmY2b1zuVY3L2UZBjTXOy-fmicb_07_01642_body.txt.ann.json',
 'arTAOoEf2T_BgxJIIfgj90xBZtNC-summer_activity_patterns_for_mosses_and_lichens_in_maritime_antarctica_body.txt.ann.json',
 'av3LD7CT3RRbishmjI6XlngOM3W0-s42003_018_0260_y_body.txt.ann.json',
 'aAtJPflG7v2rJRJ2xeyAEo18WoOe-Fraser2018_Article_EvidenceOfPlantAndAnimalCommun_body.txt.ann.json',
 'aOuKiUwqR9jgpnvU3Y.f5CsugUFO-source_body.txt.ann.json',
 'aK7fzUwfIjMZ4QvOrrj0G6Le0oWu-fmicb_10_01018_body.txt.ann.json']

In [79]:
txtfiles = [x for x in os.listdir('FraserSet/BodyOnly/Clean') if x.endswith('.txt')]
txtfiles

['fmicb-07-01642.txt',
 's42003-018-0260-y.txt',
 'summer-activity-patterns-for-mosses-and-lichens-in-maritime-antarctica.txt',
 'source.txt',
 'fmicb-10-01018.txt',
 'test.txt',
 'Archer2017_Article_EndolithicMicrobialDiversityIn.txt',
 'Fraser2018_Article_EvidenceOfPlantAndAnimalCommun.txt']

In [15]:
filename=JSONfiles[1]
file = open('tagtog/BodyOnly/'+filename)
annotatedDocJSON = file.read()
file.close()
annotations = json.loads(annotatedDocJSON)
filename = re.sub('.txt.ann.json','.txt',filename[29:])
filename = difflib.get_close_matches(filename, txtfiles, n=1)[0]
# filename = re.sub('_', '-', filename)
file = open('FraserSet/BodyOnly/Clean/'+filename)
document = file.read()
file.close()

In [16]:
doc = nlp_lg(document)

## Get annotated entities

In [20]:
def getEntitiesandRelations(JSONfolder, textFolder, i):
# Isolate only JSON and text files in respective folders
    JSONfiles = [x for x in os.listdir(JSONfolder) if x.endswith('.json')]
    txtfiles = [x for x in os.listdir(textFolder) if x.endswith('.txt')]

# Open JSON file and matching text file
    filename=JSONfiles[i]
    file = open(JSONfolder+'/'+filename)
    annotatedDocJSON = file.read()
    file.close()
    annotations = json.loads(annotatedDocJSON)
    filename = re.sub('.txt.ann.json','.txt',filename[29:])
    filename = difflib.get_close_matches(filename, txtfiles, n=1)[0]
    file = open(textFolder+'/'+filename)
    document = file.read()
    file.close()
    
#     print(filename)
    doc = nlp_lg(document)
#     print('tokenised', filename)
      
# Extract relevant information from JSON files for entities
    entities = pd.DataFrame(annotations['entities'], columns=['classId', 'offsets'])
    entities['Start_Index']= [X[0].get('start') for X in entities['offsets']]
    entities['Text']= [X[0].get('text') for X in entities['offsets']]
    entities['End_Index']= [len(X[0].get('text'))+X[0].get('start') for X in entities['offsets']]
    entities = entities[['Text', 'Start_Index', 'End_Index','classId']]
#     entities['doctext']= [document[entities.loc[X]['Start_Index']:
#                                         entities.loc[X]['End_Index']] for X in entities.index]

# The annotated files were somehow growing slightly out of sync with the orginal document when compared.
# The following lines ensure the annotation indicies are corrected to match the otginal document 
    entities['offsetR']= [entities.loc[X]['Start_Index']-document[:entities.loc[X]['End_Index']].rfind(entities.loc[X]['Text'])for X in entities.index]
    entities['offsetL']= [document[entities.loc[X]['Start_Index']:].find(entities.loc[X]['Text'])for X in entities.index]
    entities['offsetR']= [99999999 if X ==-1 else X for X in entities['offsetR']]
    entities['offsetL']= [99999999 if X ==-1 else X for X in entities['offsetL']]
    entities['adj_Start'] = [entities.loc[X]['Start_Index']-entities.loc[X]['offsetR']
                             if entities.loc[X]['offsetR']<entities.loc[X]['offsetL']
                             else entities.loc[X]['Start_Index']+entities.loc[X]['offsetL']
                                   for X in entities.index]
    entities['adj_End'] = entities['End_Index']- entities['Start_Index'] +entities['adj_Start']

    positionDict = dict(zip(entities.Start_Index, entities.adj_Start)) #used to adjust relationship indicies

    entities['adj_doctext']= [document[entities.loc[X]['adj_Start']:
                                        entities.loc[X]['adj_End']] for X in entities.index]
    entities['Start_Token']=entities.adj_Start.apply(lambda x: get_token_num_for_char(doc, x))
#     print('calculating end tokens for', filename)
    entities['End_Token']=[get_token_num_for_end_char(doc, entities.loc[X]['Start_Token'], entities.loc[X]['adj_doctext']) for X in entities.index]

    entities = entities[['adj_doctext', 'Start_Token','End_Token','adj_Start', 'adj_End', 'classId']]
    entities.columns=['Text', 'Start_Token','End_Token','Start_Index','End_Index','classId']
    
    entities=entities[(entities.classId=='e_1') | (entities.classId=='e_2') ]
    
    entities['Sentence']= [doc[entities.loc[X]['Start_Token']].sent for X in entities.index]

    AbstractStart = findAbstractStart(doc)
    entities['inAbstract300'] = inAbstract(zip(entities.Start_Token, entities.End_Token), AbstractStart, doc, 300) 
    entities['inAbstract500'] = inAbstract(zip(entities.Start_Token, entities.End_Token), AbstractStart, doc, 500) 

    entities['max_TFISF'] = calculateTFISF(zip(entities.Start_Token, entities.End_Token), doc) 

    entities['Sent_Start']= [1 if doc[entities.loc[X]['Start_Token']].is_sent_start ==True 
                                     else 0 for X in entities.index]
    entities['Document'] = filename[:-4] 
    
    cols_to_downcast = ['Start_Token', 'End_Token', 'Start_Index', 'End_Index', 
                    'inAbstract300', 'inAbstract500', 'Sent_Start']
    entities[cols_to_downcast]=entities[cols_to_downcast].apply(pd.to_numeric, downcast='integer')

    
# Create dataframes of annotated species and locations 
    species = entities[entities.classId=='e_1'].drop(columns=['classId'])
    locations = entities[entities.classId=='e_2'].drop(columns=['classId'])

# Extract relevant information from JSON files for relations
    relationships = pd.DataFrame(annotations['relations'], columns = ['entities'])
    relationships['entity1-type'] = [X[0][5:8] for X in relationships['entities']]
    relationships['entity1-type'] = ['species' if X == 'e_1' else 'location' for X in relationships['entity1-type']]
    relationships['entity1-start'] = [int(X[0][9:X[0].index(",")]) for X in relationships['entities']]
    relationships['entity1-end'] = [int(X[0][X[0].index(",")+1:])+1 for X in relationships['entities']]
    relationships['entity2-type'] = [X[1][5:8] for X in relationships['entities']]
    relationships['entity2-type'] = ['species' if X == 'e_1' else 'location' for X in relationships['entity2-type']]
    relationships['entity2-start'] = [int(X[1][9:X[1].index(",")]) for X in relationships['entities']]
    relationships['entity2-end'] = [int(X[1][X[1].index(",")+1:])+1 for X in relationships['entities']]
    relationships.drop(columns='entities', inplace=True)
    
# Rearrange order of entities so species always appears on left hand side of the pair 
    relationships[['entity1-type','entity1-start','entity1-end','entity2-type','entity2-start','entity2-end']] = relationships[['entity1-type','entity1-start','entity1-end','entity2-type','entity2-start','entity2-end']].mask(
        relationships['entity1-type'] == 'location', relationships[['entity2-type','entity2-start','entity2-end','entity1-type','entity1-start','entity1-end']].values)
    
# Adjust entity indices to match document
    relationships['entity1-adjstart']=[positionDict.get(X) for X in relationships['entity1-start']]
    relationships['entity1-adjend']=relationships['entity1-end']-relationships['entity1-start']+relationships['entity1-adjstart']

    relationships['entity2-adjstart']=[positionDict.get(X) for X in relationships['entity2-start']]
    relationships['entity2-adjend']=relationships['entity2-end']-relationships['entity2-start']+relationships['entity2-adjstart']

# Insert actual text of the entities into the relationships dataframe
    relationships['entity1']= [document[relationships.loc[X]['entity1-adjstart']:
                                        relationships.loc[X]['entity1-adjend']] for X in relationships.index]
    relationships['entity2']= [document[relationships.loc[X]['entity2-adjstart']:
                                        relationships.loc[X]['entity2-adjend']] for X in relationships.index]
    
#Add token numbers    
    relationships['Start_Token_Species']=relationships['entity1-adjstart'].apply(
        lambda x: get_token_num_for_char(doc, x))
    relationships['End_Token_Species']=[get_token_num_for_end_char(
        doc, relationships.loc[X]['Start_Token_Species'], relationships.loc[X]['entity1']) for X in relationships.index]
    relationships['Start_Token_Location']=relationships['entity2-adjstart'].apply(
        lambda x: get_token_num_for_char(doc, x))
    relationships['End_Token_Location']=[get_token_num_for_end_char(
        doc, relationships.loc[X]['Start_Token_Location'], relationships.loc[X]['entity2']) for X in relationships.index]
     
# Add Ground Truth Value for predictions
    relationships['Tagged_Relationship'] = 1
    
# Tidy the whole thing up
    relationships.drop(columns=['entity1-type', 'entity2-type','entity1-start','entity1-end','entity2-start',
                                'entity2-end'], inplace = True)
    relationships.columns=['Start_Index_Species', 'End_Index_Species','Start_Index_Location', 'End_Index_Location', 
                           'Species', 'Location', 'Start_Token_Species','End_Token_Species','Start_Token_Location',
                           'End_Token_Location','Tagged_Relationship']
    relationships=relationships[['Species', 'Start_Token_Species','End_Token_Species','Start_Index_Species', 
                                 'End_Index_Species', 'Location','Start_Token_Location','End_Token_Location',
                                 'Start_Index_Location', 'End_Index_Location','Tagged_Relationship']]

# return three dataframes
    return species, locations, relationships

In [21]:
def calculateTFISF(tokens, doc):
    results = []
    numSentsinDoc = len(list(doc.sents))
    for startToken,endToken in tokens:
        maxtfisf = 0
        for tkn in range(startToken,endToken):
            if doc[tkn].is_alpha:
                tf = len([1 for word in doc[tkn].sent if doc[tkn].text == str(word)])
                isf = np.log(
                    numSentsinDoc/len([1 for sent in doc.sents if doc[tkn].text in str(sent)]))
                tfisf = tf*isf
                maxtfisf = max(maxtfisf, tfisf)
        results.append(maxtfisf)
    return results

In [22]:
def findAbstractStart(doc):
    matcher = Matcher(nlp_lg.vocab)
    pattern = [
        [{"LOWER": "abstract"}]
    ]
    matcher.add("abstract_pattern", pattern)
    matches =matcher(doc)
    if len(matches) >0:
        AbstractToken = matches[0][1]
    else: AbstractToken =-1
    return AbstractToken

In [23]:
def inAbstract(tokens, AbstractStart, doc, limit):
    if AbstractStart==-1:
        return np.zeros(len(list(tokens)))
    else:
        results = []
    
    for startToken,endToken in tokens:
        matcher = Matcher(nlp_lg.vocab)
        pattern = []
        for tkn in range (startToken,endToken):
            pattern.append({"TEXT": doc[tkn].text})
        patterns = [pattern]
        matcher.add("species_pattern", patterns)
        matches =matcher(doc)
        idx = [match[1] for match in matches if match[1] in range(AbstractStart,AbstractStart+limit)]
        if len(idx)>0:
            results.append(1)
        else: results.append(0)
    return results

In [24]:
def get_token_num_for_char(doc, start_idx):
    for i, token in enumerate(doc):
        if start_idx > token.idx:
            continue
        if start_idx == token.idx:
            return i
        if start_idx < token.idx:
            return i - 1

In [25]:
def get_token_num_for_end_char(doc, start_token, text):
    end_token = start_token+1
    span=doc[start_token:end_token]
#     print(span.text, start_token, end_token, text)
    while len(span.text)< len(text):
        end_token+=1
        span=doc[start_token:end_token]
#         print('looping', span.text, start_token, end_token, text)
    return end_token

In [71]:
speciesAnnotated, locationsAnnotated, relationsAnnotated = getEntitiesandRelations(
            'tagtog/BodyOnly', 'FraserSet/BodyOnly/Clean', 2)

In [90]:
speciesAnnotated

,Text,Start_Token,End_Token,Start_Index,End_Index,Sentence,inAbstract300,inAbstract500,max_TFISF,Sent_Start,Document
60,Umbilicaria aprina Nyl.,1075,1079,5800,5823,"(Annual, activity, (, %, of, total, time, ), r...",0,0,5.814131,0,summer-activity-patterns-for-mosses-and-lichen...
64,Usnea aurantiaco-atra,1118,1122,5960,5981,"(2011, ), to, 45.6, %, for, Usnea, aurantiaco,...",0,0,4.715518,0,summer-activity-patterns-for-mosses-and-lichen...
104,Deschampsia antarctica,1941,1943,10374,10396,"(In, addition, ,, both, Antarctic, phanerogams...",0,0,5.814131,0,summer-activity-patterns-for-mosses-and-lichen...
105,Colobanthus quitensis,1946,1948,10407,10428,"(In, addition, ,, both, Antarctic, phanerogams...",0,0,5.814131,0,summer-activity-patterns-for-mosses-and-lichen...
108,Brachythecium austrosalebrosum,1996,1998,10668,10698,"(At, sites, with, a, regular, water, supply, ,...",0,0,4.204693,0,summer-activity-patterns-for-mosses-and-lichen...
109,Pohlia nutans,2007,2009,10720,10733,"(Kindb, ., ,, Pohlia, nutans, (, Hedw, ., ))",0,0,5.814131,0,summer-activity-patterns-for-mosses-and-lichen...
110,Bryum pseudotriquetrum,2016,2018,10750,10772,"(Lindb, ., ,, Bryum, pseudotriquetrum, (, Hedw...",0,0,4.715518,0,summer-activity-patterns-for-mosses-and-lichen...
111,Sanionia uncinata,2030,2032,10811,10828,"(and, Sanionia, uncinata, (, Hedw, ., ))",0,0,5.814131,0,summer-activity-patterns-for-mosses-and-lichen...
113,Usnea spp.,2053,2056,10931,10941,"(Dry, habitats, on, rock, and, scree, are, mai...",0,0,11.628261,0,summer-activity-patterns-for-mosses-and-lichen...
114,Umbilicaria spp.,2057,2060,10946,10962,"(Dry, habitats, on, rock, and, scree, are, mai...",0,0,11.628261,0,summer-activity-patterns-for-mosses-and-lichen...


In [88]:
JSONfolder = 'tagtog/BodyOnly'
textFolder = 'FraserSet/BodyOnly/Clean'

jFiles = [x for x in os.listdir(JSONfolder) if x.endswith('.json')]
tFiles = [x for x in os.listdir(textFolder) if x.endswith('.txt')]

filename = re.sub('.txt.ann.json','.txt',jFiles[2][29:])
filename = difflib.get_close_matches(filename, tFiles, n=1)[0]
        
speciescsvfilename = re.sub('.txt', '-species.csv', filename)
speciesExtracted=pd.read_csv(textFolder+'/'+speciescsvfilename,index_col=0 )
speciesExtracted=speciesExtracted[['Found as', 'Full name', 'Start', 'End','PositionOfFirstToken']]
speciesExtracted.columns=['Found_as', 'Full_name', 'Start_Token', 'End_Token', 'Start_Index']
speciesExtracted['End_Index']=[len(speciesExtracted.loc[X]['Found_as'])+speciesExtracted.loc[X]['Start_Index'] for X in speciesExtracted.index]

In [89]:
speciesExtracted

,Found_as,Full_name,Start_Token,End_Token,Start_Index,End_Index
0,Umbilicaria aprina,Umbilicaria aprina,1075,1077,5800,5818
1,Usnea,Usnea,1118,1119,5960,5965
2,Deschampsia antarctica,Deschampsia antarctica,1941,1943,10374,10396
3,Colobanthus quitensis,Colobanthus quitensis,1946,1948,10407,10428
4,Brachythecium austrosalebrosum,Brachythecium austrosalebrosum,1996,1998,10668,10698
5,Pohlia nutans,Pohlia nutans,2007,2009,10720,10733
6,Bryum pseudotriquetrum,Bryum pseudotriquetrum,2016,2018,10750,10772
7,Sanionia uncinata,Sanionia uncinata,2030,2032,10811,10828
8,Usnea,Usnea,2053,2054,10931,10936
9,Umbilicaria,Umbilicaria,2057,2058,10946,10957


In [73]:
locationsAnnotated.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 138 entries, 2 to 329
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Text           138 non-null    object 
 1   Start_Token    138 non-null    int16  
 2   End_Token      138 non-null    int16  
 3   Start_Index    138 non-null    int32  
 4   End_Index      138 non-null    int32  
 5   Sentence       138 non-null    object 
 6   inAbstract300  138 non-null    int8   
 7   inAbstract500  138 non-null    int8   
 8   max_TFISF      138 non-null    float64
 9   Sent_Start     138 non-null    int8   
 10  Document       138 non-null    object 
dtypes: float64(1), int16(2), int32(2), int8(3), object(3)
memory usage: 7.4+ KB


In [75]:
relationsAnnotated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Species               48 non-null     object
 1   Start_Token_Species   48 non-null     int64 
 2   End_Token_Species     48 non-null     int64 
 3   Start_Index_Species   48 non-null     int64 
 4   End_Index_Species     48 non-null     int64 
 5   Location              48 non-null     object
 6   Start_Token_Location  48 non-null     int64 
 7   End_Token_Location    48 non-null     int64 
 8   Start_Index_Location  48 non-null     int64 
 9   End_Index_Location    48 non-null     int64 
 10  Tagged_Relationship   48 non-null     int64 
dtypes: int64(9), object(2)
memory usage: 4.2+ KB


### All docs in a folder

In [60]:
def getEntityMatches (JSONfolder, textFolder):
    allSpecMatches = pd.DataFrame()
    allLocMatches = pd.DataFrame()
    allRelAnnotated = pd.DataFrame()

    jFiles = [x for x in os.listdir(JSONfolder) if x.endswith('.json')]
    tFiles = [x for x in os.listdir(textFolder) if x.endswith('.txt')]

    
    for i in range(len(jFiles)):
        speciesAnnotated, locationsAnnotated, relationsAnnotated = getEntitiesandRelations(
            JSONfolder, textFolder, i)
        
        speciesAnnotated=speciesAnnotated[['Text','Start_Token','End_Token', 'Start_Index','End_Index']]
        locationsAnnotated=locationsAnnotated[['Text','Start_Token','End_Token', 'Start_Index','End_Index']]

        filename = re.sub('.txt.ann.json','.txt',jFiles[i][29:])
        filename = difflib.get_close_matches(filename, tFiles, n=1)[0]
        
        speciescsvfilename = re.sub('.txt', '-species.csv', filename)
        speciesExtracted=pd.read_csv(textFolder+'/'+speciescsvfilename,index_col=0 )
        speciesExtracted=speciesExtracted[['Found as', 'Full name', 'Start', 'End','PositionOfFirstToken']]
        speciesExtracted.columns=['Found_as', 'Full_name', 'Start_Token', 'End_Token', 'Start_Index']
        speciesExtracted['End_Index']=[len(speciesExtracted.loc[X]['Found_as'])+speciesExtracted.loc[X]['Start_Index'] for X in speciesExtracted.index]
        
        matchesSpecies = pd.merge(speciesExtracted, speciesAnnotated, how='outer')
        matchesSpecies['Match']=matchesSpecies.Found_as == matchesSpecies.Text
        matchesSpecies['Document']=filename[:-4]
        
        locationscsvfilename = re.sub('.txt', '-locations.csv', filename)
        locationsExtracted=pd.read_csv(textFolder+'/'+locationscsvfilename,index_col=0)
        
        matchesLocations = pd.merge(locationsExtracted, locationsAnnotated, how='outer')
        matchesLocations.rename(columns = {'Location':'ExtractedLocation','Text':'TaggedLocation'}, inplace=True)
        matchesLocations['Document']=filename[:-4]
#         asBool = ['NZGazAnt','NZGaz', 'ScarNZ', 'ScarGlobal', 'GeoNamesNZ', 'GeoNamesAnt','GeoNames',
#           'exactMatch', 'Close_Match','PartialMatch', 'inNZ', 'inAntarctica','Found']
#         asObj = ['Close_Match_NZGazAnt', 'Close_Match_NZGaz','Close_Match_ScarNZ', 
#           'Close_Match_ScarGlobal','Close_Match_GeoNamesNZ', 'Close_Match_GeoNamesAnt',
#           'PartialMatch_NZGazAnt', 'PartialMatch_NZGaz', 'PartialMatch_ScarNZ','PartialMatch_ScarGlobal', 
#           'PartialMatch_GeoNamesNZ','PartialMatch_GeoNamesAnt']
#         for col in asBool:
#             matchesLocations[col] = matchesLocations[col].astype(bool)
#         for col in asObj:
#             matchesLocations[col] = matchesLocations[col].astype(object)
       
        
#         relationsAnnotated['Relationship']=1
        relationsAnnotated['Document']=filename[:-4]  
        
        allSpecMatches = pd.concat([allSpecMatches, matchesSpecies])
        allLocMatches = pd.concat([allLocMatches, matchesLocations])
        allRelAnnotated = pd.concat([allRelAnnotated, relationsAnnotated])
        
    allSpecMatches.reset_index(drop=True, inplace=True)
    allLocMatches.reset_index(drop=True, inplace=True)
    allRelAnnotated.reset_index(drop=True, inplace=True)
    
    return allSpecMatches, allLocMatches, allRelAnnotated


In [27]:
def createMatrixofConfirmedEntities (JSONfolder, textFolder):
    allPossibleRelationships = pd.DataFrame()

    jFiles = [x for x in os.listdir(JSONfolder) if x.endswith('.json')]
    tFiles = [x for x in os.listdir(textFolder) if x.endswith('.txt')]
    
    for i in range(len(jFiles)):
        speciesAnnotated, locationsAnnotated, relationsAnnotated = getEntitiesandRelations(
            JSONfolder, textFolder, i)
        
        filename = re.sub('.txt.ann.json','.txt',jFiles[i][29:])
        filename = difflib.get_close_matches(filename, tFiles, n=1)[0]
        
        possibleRelationships = pd.merge(speciesAnnotated.assign(key=0), locationsAnnotated.assign(key=0),
                                         on=['key','Document'], 
                                         suffixes=('_Species', '_Location')).drop('key', axis=1)
        possibleRelationships.rename(columns={'Text_Species':'Species', 'Text_Location':'Location'}, inplace=True)
#         possibleRelationships['Document']=filename[:-4] 
        relationsAnnotated['Document']=filename[:-4]  
        
        oneDocRelationships = pd.merge(possibleRelationships, relationsAnnotated, how='left')
        oneDocRelationships.Tagged_Relationship.fillna(0, inplace=True)
        oneDocRelationships.Tagged_Relationship = oneDocRelationships.Tagged_Relationship.astype('int8')
               
        allPossibleRelationships = pd.concat([allPossibleRelationships, oneDocRelationships])
                
    allPossibleRelationships.reset_index(drop=True, inplace=True)
    
    return allPossibleRelationships


In [12]:
filename = 'summer-activity-patterns-for-mosses-and-lichens-in-maritime-antarctica.txt'
file = open('FraserSet/BodyOnly/Clean/'+filename)
document = file.read()
file.close()
doc = nlp_lg(document)

In [32]:
possibleRelationships = pd.merge(speciesAnnotated.assign(key=0), locationsAnnotated.assign(key=0), on=['key','Document'], 
                               suffixes=('_Species', '_Location')).drop('key', axis=1)
possibleRelationships.rename(columns={'Text_Species':'Species', 'Text_Location':'Location'}, inplace=True)
# possibleRelationships['Document']=filename[:-4]  

In [33]:
possibleRelationships.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7590 entries, 0 to 7589
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Species                 7590 non-null   object 
 1   Start_Token_Species     7590 non-null   int16  
 2   End_Token_Species       7590 non-null   int16  
 3   Start_Index_Species     7590 non-null   int32  
 4   End_Index_Species       7590 non-null   int32  
 5   Sentence_Species        7590 non-null   object 
 6   inAbstract300_Species   7590 non-null   int8   
 7   inAbstract500_Species   7590 non-null   int8   
 8   max_TFISF_Species       7590 non-null   float64
 9   Sent_Start_Species      7590 non-null   int8   
 10  Document                7590 non-null   object 
 11  Location                7590 non-null   object 
 12  Start_Token_Location    7590 non-null   int16  
 13  End_Token_Location      7590 non-null   int16  
 14  Start_Index_Location    7590 non-null   

In [100]:
possibleRelationships.head()

,Species,Start_Token_Species,End_Token_Species,Start_Index_Species,End_Index_Species,Sentence_Species,inAbstract300_Species,inAbstract500_Species,max_TFISF_Species,Sent_Start_Species,...,Location,Start_Token_Location,End_Token_Location,Start_Index_Location,End_Index_Location,Sentence_Location,inAbstract300_Location,inAbstract500_Location,max_TFISF_Location,Sent_Start_Location
0,Umbilicaria aprina Nyl.,1075,1079,5800,5823,"(Annual, activity, (, %, of, total, time, ), r...",0,0,5.814131,0,...,Antarctica,9,10,60,70,"(Summer, activity, patterns, for, mosses, and,...",1,1,2.869692,0
1,Umbilicaria aprina Nyl.,1075,1079,5800,5823,"(Annual, activity, (, %, of, total, time, ), r...",0,0,5.814131,0,...,Kiel,38,39,235,239,"(Summer, activity, patterns, for, mosses, and,...",0,0,5.814131,0
2,Umbilicaria aprina Nyl.,1075,1079,5800,5823,"(Annual, activity, (, %, of, total, time, ), r...",0,0,5.814131,0,...,Germany,40,41,241,248,"(Summer, activity, patterns, for, mosses, and,...",0,0,5.120983,0
3,Umbilicaria aprina Nyl.,1075,1079,5800,5823,"(Annual, activity, (, %, of, total, time, ), r...",0,0,5.814131,0,...,Hamilton,52,53,312,320,"(Summer, activity, patterns, for, mosses, and,...",0,0,5.814131,0
4,Umbilicaria aprina Nyl.,1075,1079,5800,5823,"(Annual, activity, (, %, of, total, time, ), r...",0,0,5.814131,0,...,New Zealand,54,56,322,333,"(Summer, activity, patterns, for, mosses, and,...",0,0,5.814131,0


In [119]:
oneDocRelationships = pd.merge(possibleRelationships, relationsAnnotated, how='left')
oneDocRelationships.Tagged_Relationship.fillna(0, inplace=True)

In [120]:
oneDocRelationships.Tagged_Relationship.value_counts()

0.0    7542
1.0      48
Name: Tagged_Relationship, dtype: int64

In [121]:
oneDocRelationships.head()

,Species,Start_Token_Species,End_Token_Species,Start_Index_Species,End_Index_Species,Sentence_Species,inAbstract300_Species,inAbstract500_Species,max_TFISF_Species,Sent_Start_Species,...,Start_Token_Location,End_Token_Location,Start_Index_Location,End_Index_Location,Sentence_Location,inAbstract300_Location,inAbstract500_Location,max_TFISF_Location,Sent_Start_Location,Tagged_Relationship
0,Umbilicaria aprina Nyl.,1075,1079,5800,5823,"(Annual, activity, (, %, of, total, time, ), r...",0,0,5.814131,0,...,9,10,60,70,"(Summer, activity, patterns, for, mosses, and,...",1,1,2.869692,0,0.0
1,Umbilicaria aprina Nyl.,1075,1079,5800,5823,"(Annual, activity, (, %, of, total, time, ), r...",0,0,5.814131,0,...,38,39,235,239,"(Summer, activity, patterns, for, mosses, and,...",0,0,5.814131,0,0.0
2,Umbilicaria aprina Nyl.,1075,1079,5800,5823,"(Annual, activity, (, %, of, total, time, ), r...",0,0,5.814131,0,...,40,41,241,248,"(Summer, activity, patterns, for, mosses, and,...",0,0,5.120983,0,0.0
3,Umbilicaria aprina Nyl.,1075,1079,5800,5823,"(Annual, activity, (, %, of, total, time, ), r...",0,0,5.814131,0,...,52,53,312,320,"(Summer, activity, patterns, for, mosses, and,...",0,0,5.814131,0,0.0
4,Umbilicaria aprina Nyl.,1075,1079,5800,5823,"(Annual, activity, (, %, of, total, time, ), r...",0,0,5.814131,0,...,54,56,322,333,"(Summer, activity, patterns, for, mosses, and,...",0,0,5.814131,0,0.0


In [122]:
oneDocRelationships.to_csv('oneDocRelationships.csv')

In [135]:
allPossibleRelationships = createMatrixofConfirmedEntities('tagtog/BodyOnly','FraserSet/BodyOnly/Clean')

In [136]:
allPossibleRelationships.head()

,Species,Start_Token_Species,End_Token_Species,Start_Index_Species,End_Index_Species,Sentence_Species,inAbstract300_Species,inAbstract500_Species,max_TFISF_Species,Sent_Start_Species,...,Start_Token_Location,End_Token_Location,Start_Index_Location,End_Index_Location,Sentence_Location,inAbstract300_Location,inAbstract500_Location,max_TFISF_Location,Sent_Start_Location,Tagged_Relationship
0,Hemichloris antarctica,2566,2568,13939,13961,"(Free, -, living, algal, cells, resembling, He...",0,0,4.267364,0,...,9,12,65,84,"(Endolithic, microbial, diversity, in, sandsto...",1,1,3.420066,0,0
1,Hemichloris antarctica,2566,2568,13939,13961,"(Free, -, living, algal, cells, resembling, He...",0,0,4.267364,0,...,13,14,86,96,"(Endolithic, microbial, diversity, in, sandsto...",1,1,3.420066,0,0
2,Hemichloris antarctica,2566,2568,13939,13961,"(Free, -, living, algal, cells, resembling, He...",0,0,4.267364,0,...,81,82,436,442,"(Published, online, :, 23, August, 2016, Ó, Sp...",0,0,5.365976,0,0
3,Hemichloris antarctica,2566,2568,13939,13961,"(Free, -, living, algal, cells, resembling, He...",0,0,4.267364,0,...,82,83,443,453,"(Published, online, :, 23, August, 2016, Ó, Sp...",0,0,5.365976,0,0
4,Hemichloris antarctica,2566,2568,13939,13961,"(Free, -, living, algal, cells, resembling, He...",0,0,4.267364,0,...,93,94,533,543,"(Abstract, Cryptic, microbial, communities, de...",1,1,3.420066,0,0


In [137]:
allPossibleRelationships.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21418 entries, 0 to 21417
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Species                 21418 non-null  object 
 1   Start_Token_Species     21418 non-null  int16  
 2   End_Token_Species       21418 non-null  int16  
 3   Start_Index_Species     21418 non-null  int32  
 4   End_Index_Species       21418 non-null  int32  
 5   Sentence_Species        21418 non-null  object 
 6   inAbstract300_Species   21418 non-null  int8   
 7   inAbstract500_Species   21418 non-null  int8   
 8   max_TFISF_Species       21418 non-null  float64
 9   Sent_Start_Species      21418 non-null  int8   
 10  Document                21418 non-null  object 
 11  Location                21418 non-null  object 
 12  Start_Token_Location    21418 non-null  int16  
 13  End_Token_Location      21418 non-null  int16  
 14  Start_Index_Location    21418 non-null

In [138]:
allPossibleRelationships.Tagged_Relationship.value_counts()

0    21289
1      129
Name: Tagged_Relationship, dtype: int64

In [139]:
allPossibleRelationships.to_csv('allPossibleRelationships.csv')

In [61]:
SpeciesMatches, LocationMatches, Relations = getEntityMatches('tagtog/BodyOnly','FraserSet/BodyOnly/Clean')

In [66]:
SpeciesMatches.tail()

,Found_as,Full_name,Start_Token,End_Token,Start_Index,End_Index,Text,Match,Document
410,NaN,NaN,8954,8955,46870,46880,Phormidium,False,fmicb-10-01018
411,NaN,NaN,8976,8978,46992,47015,Plectolyngbya hodgsonii,False,fmicb-10-01018
412,NaN,NaN,8979,8981,47020,47030,L. frigida,False,fmicb-10-01018
413,NaN,NaN,8999,9000,47124,47136,Pectolyngbya,False,fmicb-10-01018
414,NaN,NaN,9081,9083,47612,47622,L. frigida,False,fmicb-10-01018


In [65]:
SpeciesMatches[SpeciesMatches.Text.isna()]

,Found_as,Full_name,Start_Token,End_Token,Start_Index,End_Index,Text,Match,Document
0,Hemichloris antarctica,Hemichloris antarctica,2565,2567,13939,13961,NaN,False,Archer2017_Article_EndolithicMicrobialDiversityIn
1,Diplosphaera,Diplosphaera,3434,3435,18847,18859,NaN,False,Archer2017_Article_EndolithicMicrobialDiversityIn
2,Hemichloris antarctica,Hemichloris antarctica,3474,3476,19057,19079,NaN,False,Archer2017_Article_EndolithicMicrobialDiversityIn
3,Trebouxia,Trebouxia,3488,3489,19137,19146,NaN,False,Archer2017_Article_EndolithicMicrobialDiversityIn
4,Lecidea,Lecidea,3531,3532,19380,19387,NaN,False,Archer2017_Article_EndolithicMicrobialDiversityIn
...,...,...,...,...,...,...,...,...,...
329,Pseudanabaena,Pseudanabaena,8849,8850,46352,46365,NaN,False,fmicb-10-01018
330,Phormidium,Phormidium,8852,8853,46371,46381,NaN,False,fmicb-10-01018
331,Phormidium,Phormidium,8943,8944,46870,46880,NaN,False,fmicb-10-01018
332,L. frigida,Leptolyngbya frigida,8968,8970,47020,47030,NaN,False,fmicb-10-01018


In [67]:
SpeciesMatches[SpeciesMatches.Found_as.isna()]

,Found_as,Full_name,Start_Token,End_Token,Start_Index,End_Index,Text,Match,Document
10,NaN,NaN,2566,2568,13939,13961,Hemichloris antarctica,False,Archer2017_Article_EndolithicMicrobialDiversityIn
11,NaN,NaN,3223,3224,17566,17578,Leptolyngbya,False,Archer2017_Article_EndolithicMicrobialDiversityIn
12,NaN,NaN,3435,3438,18847,18863,Diplosphaera sp.,False,Archer2017_Article_EndolithicMicrobialDiversityIn
13,NaN,NaN,3475,3477,19057,19079,Hemichloris antarctica,False,Archer2017_Article_EndolithicMicrobialDiversityIn
14,NaN,NaN,3489,3490,19137,19146,Trebouxia,False,Archer2017_Article_EndolithicMicrobialDiversityIn
...,...,...,...,...,...,...,...,...,...
410,NaN,NaN,8954,8955,46870,46880,Phormidium,False,fmicb-10-01018
411,NaN,NaN,8976,8978,46992,47015,Plectolyngbya hodgsonii,False,fmicb-10-01018
412,NaN,NaN,8979,8981,47020,47030,L. frigida,False,fmicb-10-01018
413,NaN,NaN,8999,9000,47124,47136,Pectolyngbya,False,fmicb-10-01018


In [62]:
SpeciesMatches[SpeciesMatches.Start_Index==17566]

,Found_as,Full_name,Start_Token,End_Token,Start_Index,End_Index,Text,Match,Document
11,NaN,NaN,3223,3224,17566,17578,Leptolyngbya,False,Archer2017_Article_EndolithicMicrobialDiversityIn


In [1154]:
LocationMatches.head()

,ExtractedLocation,Original Tokenised Text,Start_Token,End_Token,Start_Index,End_Index,Sentence,NZGazAnt,NZGaz,ScarNZ,...,PartialMatch_ScarNZ,PartialMatch_ScarGlobal,PartialMatch_GeoNamesNZ,PartialMatch_GeoNamesAnt,PartialMatch,inNZ,inAntarctica,Found,TaggedLocation,Document
0,McMurdo Dry Valleys,the McMurdo Dry Valleys,9,12,65,84,Endolithic microbial diversity in sandstone an...,True,True,True,...,NaN,NaN,NaN,NaN,NaN,True,True,True,NaN,Archer2017_Article_EndolithicMicrobialDiversityIn
1,Antarctica,Antarctica,13,14,86,96,"Antarctica Stephen D. J. Archer1,2 •",False,False,False,...,NaN,NaN,NaN,NaN,NaN,False,True,True,Antarctica,Archer2017_Article_EndolithicMicrobialDiversityIn
2,Antarctica,Antarctica,93,94,533,543,Abstract Cryptic microbial communities develop...,False,False,False,...,NaN,NaN,NaN,NaN,NaN,False,True,True,NaN,Archer2017_Article_EndolithicMicrobialDiversityIn
3,McMurdo Dry Valleys,McMurdo Dry Valleys,95,98,546,565,Abstract Cryptic microbial communities develop...,True,True,True,...,NaN,NaN,NaN,NaN,NaN,True,True,True,McMurdo Dry Valleys,Archer2017_Article_EndolithicMicrobialDiversityIn
4,Mars,Mars,274,275,1714,1718,The study demonstrates that endolithic microbi...,False,False,False,...,NaN,NaN,NaN,NaN,False,False,False,True,NaN,Archer2017_Article_EndolithicMicrobialDiversityIn


In [68]:
LocationMatches.head()

,ExtractedLocation,Original Tokenised Text,Start_Token,End_Token,Start_Index,End_Index,Sentence,NZGazAnt,NZGaz,ScarNZ,...,PartialMatch_ScarNZ,PartialMatch_ScarGlobal,PartialMatch_GeoNamesNZ,PartialMatch_GeoNamesAnt,PartialMatch,inNZ,inAntarctica,Found,TaggedLocation,Document
0,McMurdo Dry Valleys,the McMurdo Dry Valleys,9,12,65,84,Endolithic microbial diversity in sandstone an...,True,True,True,...,NaN,NaN,NaN,NaN,NaN,True,True,True,McMurdo Dry Valleys,Archer2017_Article_EndolithicMicrobialDiversityIn
1,Antarctica,Antarctica,13,14,86,96,"Antarctica Stephen D. J. Archer1,2 •",False,False,False,...,NaN,NaN,NaN,NaN,NaN,False,True,True,Antarctica,Archer2017_Article_EndolithicMicrobialDiversityIn
2,Antarctica,Antarctica,93,94,533,543,Abstract Cryptic microbial communities develop...,False,False,False,...,NaN,NaN,NaN,NaN,NaN,False,True,True,Antarctica,Archer2017_Article_EndolithicMicrobialDiversityIn
3,McMurdo Dry Valleys,McMurdo Dry Valleys,95,98,546,565,Abstract Cryptic microbial communities develop...,True,True,True,...,NaN,NaN,NaN,NaN,NaN,True,True,True,McMurdo Dry Valleys,Archer2017_Article_EndolithicMicrobialDiversityIn
4,Mars,Mars,274,275,1714,1718,The study demonstrates that endolithic microbi...,False,False,False,...,NaN,NaN,NaN,NaN,False,False,False,True,Mars,Archer2017_Article_EndolithicMicrobialDiversityIn


In [92]:
LocationMatches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 553 entries, 0 to 552
Data columns (total 34 columns):
ExtractedLocation           250 non-null object
Original Tokenised Text     250 non-null object
Start_Token                 553 non-null int64
End_Token                   553 non-null int64
Start_Index                 553 non-null int64
End_Index                   553 non-null int64
Sentence                    250 non-null object
NZGazAnt                    250 non-null object
NZGaz                       250 non-null object
ScarNZ                      250 non-null object
ScarGlobal                  250 non-null object
GeoNamesNZ                  250 non-null object
GeoNamesAnt                 250 non-null object
GeoNames                    250 non-null object
exactMatch                  250 non-null object
Close_Match_NZGazAnt        59 non-null object
Close_Match_NZGaz           59 non-null object
Close_Match_ScarNZ          3 non-null object
Close_Match_ScarGlobal      4 non-null 

In [64]:
LocationMatches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 676 entries, 0 to 675
Data columns (total 34 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ExtractedLocation         250 non-null    object 
 1   Original Tokenised Text   250 non-null    object 
 2   Start_Token               676 non-null    int64  
 3   End_Token                 676 non-null    int64  
 4   Start_Index               676 non-null    int64  
 5   End_Index                 676 non-null    int64  
 6   Sentence                  250 non-null    object 
 7   NZGazAnt                  250 non-null    object 
 8   NZGaz                     250 non-null    object 
 9   ScarNZ                    250 non-null    object 
 10  ScarGlobal                250 non-null    object 
 11  GeoNamesNZ                250 non-null    object 
 12  GeoNamesAnt               250 non-null    object 
 13  GeoNames                  250 non-null    object 
 14  exactMatch

In [93]:
LocationMatches.columns

Index(['ExtractedLocation', 'Original Tokenised Text', 'Start_Token',
       'End_Token', 'Start_Index', 'End_Index', 'Sentence', 'NZGazAnt',
       'NZGaz', 'ScarNZ', 'ScarGlobal', 'GeoNamesNZ', 'GeoNamesAnt',
       'GeoNames', 'exactMatch', 'Close_Match_NZGazAnt', 'Close_Match_NZGaz',
       'Close_Match_ScarNZ', 'Close_Match_ScarGlobal',
       'Close_Match_GeoNamesNZ', 'Close_Match_GeoNamesAnt', 'Close_Match',
       'PartialMatch_NZGazAnt', 'PartialMatch_NZGaz', 'PartialMatch_ScarNZ',
       'PartialMatch_ScarGlobal', 'PartialMatch_GeoNamesNZ',
       'PartialMatch_GeoNamesAnt', 'PartialMatch', 'inNZ', 'inAntarctica',
       'Found', 'TaggedLocation', 'Document'],
      dtype='object')

In [43]:
asStrings =['ExtractedLocation', 'Original Tokenised Text','Sentence','TaggedLocation', 'Document']
asInts = ['Start_Token','End_Token', 'Start_Index', 'End_Index']
asBool = ['NZGazAnt','NZGaz', 'ScarNZ', 'ScarGlobal', 'GeoNamesNZ', 'GeoNamesAnt','GeoNames',
          'exactMatch', 'Close_Match_NZGazAnt', 'Close_Match_NZGaz','Close_Match_ScarNZ', 
          'Close_Match_ScarGlobal','Close_Match_GeoNamesNZ', 'Close_Match_GeoNamesAnt', 'Close_Match',
          'PartialMatch_NZGazAnt', 'PartialMatch_NZGaz', 'PartialMatch_ScarNZ','PartialMatch_ScarGlobal', 
          'PartialMatch_GeoNamesNZ','PartialMatch_GeoNamesAnt', 'PartialMatch', 'inNZ', 'inAntarctica','Found']
for col in asStrings:
    LocationMatches[col] = LocationMatches[col].astype(str)
for col in asBool:
    LocationMatches[col] = LocationMatches[col].astype(bool)

In [44]:
LocationMatches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 796 entries, 0 to 795
Data columns (total 38 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ExtractedLocation         796 non-null    object 
 1   Original Tokenised Text   796 non-null    object 
 2   Start_Token               796 non-null    int64  
 3   End_Token                 796 non-null    int64  
 4   Start_Index               796 non-null    int64  
 5   End_Index                 796 non-null    int64  
 6   Sentence                  796 non-null    object 
 7   NZGazAnt                  796 non-null    bool   
 8   NZGaz                     796 non-null    bool   
 9   ScarNZ                    796 non-null    bool   
 10  ScarGlobal                796 non-null    bool   
 11  GeoNamesNZ                796 non-null    bool   
 12  GeoNamesAnt               796 non-null    bool   
 13  GeoNames                  796 non-null    bool   
 14  exactMatch

In [1100]:
Relations

,Species,entity1-start,entity1-end,Location,entity2-start,entity2-end,Relationship,Document
0,Hemichloris antarctica,13939,13961,University Valley,13219,13236,1,Archer2017_Article_EndolithicMicrobialDiversityIn
1,Leptolyngbya,17566,17578,University Valley,17639,17656,1,Archer2017_Article_EndolithicMicrobialDiversityIn
2,Diplosphaera sp.,18847,18863,McKelvey,18882,18890,1,Archer2017_Article_EndolithicMicrobialDiversityIn
3,Diplosphaera sp.,18847,18863,Miers valleys,18895,18908,1,Archer2017_Article_EndolithicMicrobialDiversityIn
4,Hemichloris antarctica,19057,19079,University Valley,18986,19003,1,Archer2017_Article_EndolithicMicrobialDiversityIn
5,Lecidea,19380,19387,University Valley,19417,19434,1,Archer2017_Article_EndolithicMicrobialDiversityIn
6,Nitrososphaera gargensis,19685,19709,McKelvey,19771,19779,1,Archer2017_Article_EndolithicMicrobialDiversityIn
7,Nitrososphaera gargensis,19685,19709,Miers valleys,19784,19797,1,Archer2017_Article_EndolithicMicrobialDiversityIn
8,Hemichloris antarctica,25155,25177,University Valley,25218,25235,1,Archer2017_Article_EndolithicMicrobialDiversityIn
9,Diplosphaera,25299,25311,Miers,25362,25367,1,Archer2017_Article_EndolithicMicrobialDiversityIn


In [1135]:
entities = pd.DataFrame(annotations['entities'], columns=['classId', 'offsets'])
entities['Start_Index']= [X[0].get('start') for X in entities['offsets']]
entities['Text']= [X[0].get('text') for X in entities['offsets']]
entities['End_Index']= [len(X[0].get('text'))+X[0].get('start') for X in entities['offsets']]
entities = entities[['Text', 'Start_Index', 'End_Index','classId']]
entities['doctext']= [document[entities.loc[X]['Start_Index']:
                                    entities.loc[X]['End_Index']] for X in entities.index]
entities['offsetR']= [entities.loc[X]['Start_Index']-document[:entities.loc[X]['End_Index']].rfind(entities.loc[X]['Text'])for X in entities.index]
entities['offsetL']= [document[entities.loc[X]['Start_Index']:].find(entities.loc[X]['Text'])for X in entities.index]
entities['offsetR']= [99999999 if X ==-1 else X for X in entities['offsetR']]
entities['offsetL']= [99999999 if X ==-1 else X for X in entities['offsetL']]
entities['adj_Start'] = [entities.loc[X]['Start_Index']-entities.loc[X]['offsetR']
                         if entities.loc[X]['offsetR']<entities.loc[X]['offsetL']
                         else entities.loc[X]['Start_Index']+entities.loc[X]['offsetL']
                               for X in entities.index]
entities['adj_End'] = entities['End_Index']- entities['Start_Index'] +entities['adj_Start']

positionDict = dict(zip(entities.Start_Index, entities.adj_Start))

entities['adj_doctext']= [document[entities.loc[X]['adj_Start']:
                                    entities.loc[X]['adj_End']] for X in entities.index]
entities = entities[['adj_doctext', 'adj_Start', 'adj_End', 'classId']]
entities.columns=['Text', 'Start_Index','End_Index','classId']
entities.head()

,Text,Start_Index,End_Index,classId
0,,-1,18,e_2
1,,-1,9,e_2
2,,-1,5,e_2
3,,-1,9,e_2
4,Antarctica,652,662,e_2


### Relationships

In [1092]:
relationships = pd.DataFrame(annotations['relations'], columns = ['entities'])
relationships['entity1-type'] = [X[0][5:8] for X in relationships['entities']]
relationships['entity1-type'] = ['species' if X == 'e_1' else 'location' for X in relationships['entity1-type']]
relationships['entity1-start'] = [int(X[0][9:X[0].index(",")]) for X in relationships['entities']]
relationships['entity1-end'] = [int(X[0][X[0].index(",")+1:])+1 for X in relationships['entities']]
relationships['entity2-type'] = [X[1][5:8] for X in relationships['entities']]
relationships['entity2-type'] = ['species' if X == 'e_1' else 'location' for X in relationships['entity2-type']]
relationships['entity2-start'] = [int(X[1][9:X[1].index(",")]) for X in relationships['entities']]
relationships['entity2-end'] = [int(X[1][X[1].index(",")+1:])+1 for X in relationships['entities']]
relationships.drop(columns='entities', inplace=True)
relationships[['entity1-type','entity1-start','entity1-end','entity2-type','entity2-start','entity2-end']] = relationships[['entity1-type','entity1-start','entity1-end','entity2-type','entity2-start','entity2-end']].mask(
    relationships['entity1-type'] == 'location', relationships[['entity2-type','entity2-start','entity2-end','entity1-type','entity1-start','entity1-end']].values)
relationships['entity1-adjstart']=[positionDict.get(X) for X in relationships['entity1-start']]
relationships['entity1-adjend']=relationships['entity1-end']-relationships['entity1-start']+relationships['entity1-adjstart']

relationships['entity2-adjstart']=[positionDict.get(X) for X in relationships['entity2-start']]
relationships['entity2-adjend']=relationships['entity2-end']-relationships['entity2-start']+relationships['entity2-adjstart']


relationships['entity1']= [document[relationships.loc[X]['entity1-adjstart']:
                                    relationships.loc[X]['entity1-adjend']] for X in relationships.index]
relationships['entity2']= [document[relationships.loc[X]['entity2-adjstart']:
                                    relationships.loc[X]['entity2-adjend']] for X in relationships.index]

relationships.drop(columns=['entity1-type', 'entity2-type','entity1-start','entity1-end','entity2-start','entity2-end'], inplace = True)
relationships.columns=['entity1-start', 'entity1-end','entity2-start', 'entity2-end', 'Species', 'Location']
relationships=relationships[['Species', 'entity1-start', 'entity1-end', 
                            'Location','entity2-start', 'entity2-end']]

relationships

,Species,entity1-start,entity1-end,Location,entity2-start,entity2-end
0,igh dispersal rates of,13939,13961,es part of this v,13219,13236
1,lising niche,17566,17578,". However, in mos",17639,17656
2,ss tolerant and,18847,18863,MDV soi,18882,18890
3,ss tolerant and,18847,18863,ing also able,18895,18908
4,been considered a mino,19057,19079,ourishes at highe,18986,19003
5,iotic i,19380,19387,"ition, in determi",19417,19434
6,"ive biotic interactions,",19685,19709,such as,19771,19779
7,"ive biotic interactions,",19685,19709,moisture and,19784,19797
8,d spectrum (in the 10.,25155,25177,at 7-derived temp,25218,25235
9,erature logg,25299,25311,CA) w,25362,25367


In [590]:
speciesAnnotated.head()

,Text,Start_Index,End_Index
56,Hemichloris antarctica,13939,13961
91,Leptolyngbya,17566,17578
108,Diplosphaera sp.,18847,18863
113,Hemichloris antarctica,19057,19079
116,Trebouxia,19137,19146


In [591]:
filename

'Archer2017_Article_EndolithicMicrobialDiversityIn.txt'

In [594]:
speciescsvfilename = re.sub('.txt', '-species.csv', filename)
speciesExtracted=pd.read_csv('FraserSet/BodyOnly/Clean/'+speciescsvfilename,index_col=0 )
speciesExtracted=speciesExtracted[['Found as', 'Full name', 'Start', 'End','PositionOfFirstToken']]
speciesExtracted.columns=['Found_as', 'Full_name', 'Start_Token', 'End_Token', 'Start_Index']
speciesExtracted['End_Index']=[len(speciesExtracted.loc[X]['Found_as'])+speciesExtracted.loc[X]['Start_Index'] for X in speciesExtracted.index]

In [595]:
speciesExtracted.head()

,Found_as,Full_name,Start_Token,End_Token,Start_Index,End_Index
0,Hemichloris antarctica,Hemichloris antarctica,2565,2567,13939,13961
1,Diplosphaera,Diplosphaera,3434,3435,18847,18859
2,Hemichloris antarctica,Hemichloris antarctica,3474,3476,19057,19079
3,Trebouxia,Trebouxia,3488,3489,19137,19146
4,Lecidea,Lecidea,3531,3532,19380,19387


In [180]:
locationscsvfilename = re.sub('.txt', '-locations.csv', filename)
locationsExtracted=pd.read_csv('FraserSet/BodyOnly/Clean/'+locationscsvfilename,index_col=0)
locationsExtracted.rename(columns = {'Start':'Start_Token','End':'End_Token','Position of first token':'Start_Index'}, inplace=True)
locationsExtracted['End_Index']=[len(locationsExtracted.loc[X]['Location'])+locationsExtracted.loc[X]['Start_Index'] for X in locationsExtracted.index]
endindexcol = locationsExtracted.End_Index
locationsExtracted.drop(columns='End_Index', inplace=True)
locationsExtracted.insert(5, 'End_Index', endindexcol)

In [181]:
locationsExtracted.head()

,Location,Original Tokenised Text,Start_Token,End_Token,Start_Index,End_Index,Sentence,NZGazAnt,NZGaz,ScarNZ,...,PartialMatch_NZGazAnt,PartialMatch_NZGaz,PartialMatch_ScarNZ,PartialMatch_ScarGlobal,PartialMatch_GeoNamesNZ,PartialMatch_GeoNamesAnt,PartialMatch,inNZ,inAntarctica,Found
0,McMurdo Dry Valleys,McMurdo Dry Valleys,14,17,97,116,Taxonomic and Functional Diversity of Soil and...,True,True,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True,True
1,Antarctica,Antarctica,287,288,1552,1562,The McMurdo Dry Valleys of Antarctica are an e...,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True,True
2,Miers Valley,the Miers Valley,350,352,1985,1997,Here we report a combined diversity and functi...,True,True,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True,True
3,McMurdo Dry Valleys,the McMurdo Dry Valleys,354,357,2005,2024,Here we report a combined diversity and functi...,True,True,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True,True
4,Antarctica,Antarctica,358,359,2028,2038,Here we report a combined diversity and functi...,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True,True


In [614]:
locationsAnnotated.head()

,Text,Start_Index,End_Index
0,McMurdo Dry Valleys,65,84
1,Antarctica,86,96
2,Berlin,436,442
3,Heidelberg,443,453
4,Antarctica,533,543


## Metrics

In [37]:
def getSpeciesMetrics(speciesMatchesDF):
    Document = []
    Tagged = []
    Extracted = []
    TruePositive = []
    FalsePositive = []
    FalseNegative = []
    Precision = []
    Recall = []
    F1 =[]
    partialMatches = []

    for doc in speciesMatchesDF.Document.unique():
        Document.append(doc)
        ActualResults = len(speciesMatchesDF[(speciesMatchesDF.Document==doc)& (speciesMatchesDF['Text'].notna())])
        PredictedResults = len(speciesMatchesDF[(speciesMatchesDF.Document==doc)& (speciesMatchesDF['Found_as'].notna())])
        Tagged.append(ActualResults)
        Extracted.append(PredictedResults)
        TP = len(speciesMatchesDF[(speciesMatchesDF.Document==doc)& (speciesMatchesDF['Found_as'].notna()) & (speciesMatchesDF['Text'].notna())])
        TruePositive.append(TP)
        FalsePositive.append(len(speciesMatchesDF[(speciesMatchesDF.Document==doc)& speciesMatchesDF['Found_as'].notna() & speciesMatchesDF['Text'].isna()]))
        FalseNegative.append(len(speciesMatchesDF[(speciesMatchesDF.Document==doc)& speciesMatchesDF['Found_as'].isna() & speciesMatchesDF['Text'].notna()]))
        notTagged = speciesMatchesDF[(speciesMatchesDF.Document ==doc) & speciesMatchesDF.Text.isna()]
        notExtracted = speciesMatchesDF[(speciesMatchesDF.Document==doc) & speciesMatchesDF.Found_as.isna()]
        closeMatches = 0
        for i in notTagged.index:
            for j in notExtracted.index:
                if notTagged.loc[i]['Start_Index']>=notExtracted.loc[j]['Start_Index'] and notTagged.loc[i]['End_Index']<=notExtracted.loc[j]['End_Index']:
                    closeMatches+=1
        for i in notExtracted.index:
            for j in notTagged.index:
                if notExtracted.loc[i]['Start_Index']>=notTagged.loc[j]['Start_Index'] and notExtracted.loc[i]['End_Index']<=notTagged.loc[j]['End_Index']:
                    closeMatches+=1
        partialMatches.append(closeMatches)

    Document.append('Total')
    ActualResults = len(speciesMatchesDF[speciesMatchesDF['Text'].notna()])
    PredictedResults = len(speciesMatchesDF[speciesMatchesDF['Found_as'].notna()])
    Tagged.append(ActualResults)
    Extracted.append(PredictedResults)
    TP=len(speciesMatchesDF[speciesMatchesDF['Found_as'].notna() & speciesMatchesDF['Text'].notna()])
    TruePositive.append(TP)
    FalsePositive.append(len(speciesMatchesDF[speciesMatchesDF['Found_as'].notna() & speciesMatchesDF['Text'].isna()]))
    FalseNegative.append(len(speciesMatchesDF[speciesMatchesDF['Found_as'].isna() & speciesMatchesDF['Text'].notna()]))
    partialMatches.append(sum(partialMatches))

    results = pd.DataFrame({'Document': Document, 'Tagged':Tagged, 'Extracted':Extracted, 'True Positives':TruePositive,
                            'False Positives':FalsePositive, 'False Negatives':FalseNegative, 'Partial Matches':partialMatches,
    #                         'Precision':Precision, 'Recall':Recall, 'F1':F1
                           })
    results['Precision']= np.round(results['True Positives']/results['Extracted'], 4)
    results['Recall']= np.round(results['True Positives']/results['Tagged'], 4)
    results['F1'] = np.round(2* ((results['Precision']*results['Recall'])/(results['Precision']+results['Recall'])), 4)
    results['Adj. Precision']= np.round((results['True Positives']+results['Partial Matches'])/results['Extracted'], 4)
    results['Adj. Recall']= np.round((results['True Positives']+results['Partial Matches'])/results['Tagged'], 4)
    results['Adj. F1'] = np.round(2* ((results['Adj. Precision']*results['Adj. Recall'])/(results['Adj. Precision']+results['Adj. Recall'])), 4)

    return results

In [36]:
def getLocationsMetrics(locationsMatchesDF):
    Document = []
    Tagged = []
    Extracted = []
    TruePositive = []
    FalsePositive = []
    FalseNegative = []
    Precision = []
    Recall = []
    F1 =[]
    partialMatches = []

    for doc in locationsMatchesDF.Document.unique():
        Document.append(doc)
        ActualResults = len(locationsMatchesDF[(locationsMatchesDF.Document==doc)& (locationsMatchesDF['TaggedLocation'].notna())])
        PredictedResults = len(locationsMatchesDF[(locationsMatchesDF.Document==doc)& (locationsMatchesDF['ExtractedLocation'].notna())])
        Tagged.append(ActualResults)
        Extracted.append(PredictedResults)
        TP = len(locationsMatchesDF[(locationsMatchesDF.Document==doc)& (locationsMatchesDF['ExtractedLocation'].notna()) & (locationsMatchesDF['TaggedLocation'].notna())])
        TruePositive.append(TP)
        FalsePositive.append(len(locationsMatchesDF[(locationsMatchesDF.Document==doc)& locationsMatchesDF['ExtractedLocation'].notna() & locationsMatchesDF['TaggedLocation'].isna()]))
        FalseNegative.append(len(locationsMatchesDF[(locationsMatchesDF.Document==doc)& locationsMatchesDF['ExtractedLocation'].isna() & locationsMatchesDF['TaggedLocation'].notna()]))
        notTagged = locationsMatchesDF[(locationsMatchesDF.Document ==doc) & locationsMatchesDF.TaggedLocation.isna()]
        notExtracted = locationsMatchesDF[(locationsMatchesDF.Document==doc) & locationsMatchesDF.ExtractedLocation.isna()]
        closeMatches = 0
        for i in notTagged.index:
            for j in notExtracted.index:
                if notTagged.loc[i]['Start_Index']>=notExtracted.loc[j]['Start_Index'] and notTagged.loc[i]['End_Index']<=notExtracted.loc[j]['End_Index']:
                    closeMatches+=1
        for i in notExtracted.index:
            for j in notTagged.index:
                if notExtracted.loc[i]['Start_Index']>=notTagged.loc[j]['Start_Index'] and notExtracted.loc[i]['End_Index']<=notTagged.loc[j]['End_Index']:
                    closeMatches+=1
        partialMatches.append(closeMatches)

    Document.append('Total')
    ActualResults = len(locationsMatchesDF[locationsMatchesDF['TaggedLocation'].notna()])
    PredictedResults = len(locationsMatchesDF[locationsMatchesDF['ExtractedLocation'].notna()])
    Tagged.append(ActualResults)
    Extracted.append(PredictedResults)
    TP=len(locationsMatchesDF[locationsMatchesDF['ExtractedLocation'].notna() & locationsMatchesDF['TaggedLocation'].notna()])
    TruePositive.append(TP)
    FalsePositive.append(len(locationsMatchesDF[locationsMatchesDF['ExtractedLocation'].notna() & locationsMatchesDF['TaggedLocation'].isna()]))
    FalseNegative.append(len(locationsMatchesDF[locationsMatchesDF['ExtractedLocation'].isna() & locationsMatchesDF['TaggedLocation'].notna()]))
    partialMatches.append(sum(partialMatches))

    results = pd.DataFrame({'Document': Document, 'Tagged':Tagged, 'Extracted':Extracted, 'True Positives':TruePositive,
                            'False Positives':FalsePositive, 'False Negatives':FalseNegative, 'Partial Matches':partialMatches,
    #                         'Precision':Precision, 'Recall':Recall, 'F1':F1
                           })
    results['Precision']= np.round(results['True Positives']/results['Extracted'], 4)
    results['Recall']= np.round(results['True Positives']/results['Tagged'], 4)
    results['F1'] = np.round(2* ((results['Precision']*results['Recall'])/(results['Precision']+results['Recall'])), 4)
    results['Adj. Precision']= np.round((results['True Positives']+results['Partial Matches'])/results['Extracted'], 4)
    results['Adj. Recall']= np.round((results['True Positives']+results['Partial Matches'])/results['Tagged'], 4)
    results['Adj. F1'] = np.round(2* ((results['Adj. Precision']*results['Adj. Recall'])/(results['Adj. Precision']+results['Adj. Recall'])), 4)

    return results

415

In [1069]:
speciesMetrics = getSpeciesMetrics(SpeciesMatches)
speciesMetrics

,Document,Tagged,Extracted,True Positives,False Positives,False Negatives,Partial Matches,Precision,Recall,F1,Adj. Precision,Adj. Recall,Adj. F1
0,Archer2017_Article_EndolithicMicrobialDiversityIn,12,10,9,1,3,1,0.9000,0.7500,0.8182,1.0000,0.8333,0.8182
1,fmicb-07-01642,13,11,10,1,3,1,0.9091,0.7692,0.8333,1.0000,0.8462,0.8333
2,summer-activity-patterns-for-mosses-and-lichen...,55,51,43,8,12,8,0.8431,0.7818,0.8113,1.0000,0.9273,0.8113
3,s42003-018-0260-y,92,91,91,0,1,0,1.0000,0.9891,0.9945,1.0000,0.9891,0.9945
4,Fraser2018_Article_EvidenceOfPlantAndAnimalCommun,3,2,2,0,1,0,1.0000,0.6667,0.8000,1.0000,0.6667,0.8000
5,source,8,8,8,0,0,0,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
6,fmicb-10-01018,83,35,30,5,53,3,0.8571,0.3614,0.5084,0.9429,0.3976,0.5084
7,Total,266,208,193,15,73,13,0.9279,0.7256,0.8144,0.9904,0.7744,0.8144


In [1070]:
locationMetrics = getLocationsMetrics(LocationMatches)
locationMetrics

,Document,Tagged,Extracted,True Positives,False Positives,False Negatives,Partial Matches,Precision,Recall,F1,Adj. Precision,Adj. Recall,Adj. F1
0,Archer2017_Article_EndolithicMicrobialDiversityIn,96,33,32,1,64,0,0.9697,0.3333,0.4961,0.9697,0.3333,0.4961
1,fmicb-07-01642,87,32,31,1,56,2,0.9688,0.3563,0.5210,1.0312,0.3793,0.5210
2,summer-activity-patterns-for-mosses-and-lichen...,138,79,78,1,60,0,0.9873,0.5652,0.7189,0.9873,0.5652,0.7189
3,s42003-018-0260-y,35,9,8,1,27,2,0.8889,0.2286,0.3637,1.1111,0.2857,0.3637
4,Fraser2018_Article_EvidenceOfPlantAndAnimalCommun,64,35,34,1,30,0,0.9714,0.5312,0.6868,0.9714,0.5312,0.6868
5,source,31,9,9,0,22,0,1.0000,0.2903,0.4500,1.0000,0.2903,0.4500
6,fmicb-10-01018,95,53,52,1,43,0,0.9811,0.5474,0.7027,0.9811,0.5474,0.7027
7,Total,546,250,244,6,302,4,0.9760,0.4469,0.6131,0.9920,0.4542,0.6131


In [69]:
speciesMetrics = getSpeciesMetrics(SpeciesMatches)
speciesMetrics

,Document,Tagged,Extracted,True Positives,False Positives,False Negatives,Partial Matches,Precision,Recall,F1,Adj. Precision,Adj. Recall,Adj. F1
0,Archer2017_Article_EndolithicMicrobialDiversityIn,12,10,0,10,12,19,0.0000,0.0000,NaN,1.9000,1.5833,1.7273
1,fmicb-07-01642,13,11,0,11,13,21,0.0000,0.0000,NaN,1.9091,1.6154,1.7500
2,summer-activity-patterns-for-mosses-and-lichen...,55,51,16,35,39,62,0.3137,0.2909,0.3019,1.5294,1.4182,1.4717
3,s42003-018-0260-y,92,91,34,57,58,114,0.3736,0.3696,0.3716,1.6264,1.6087,1.6175
4,Fraser2018_Article_EvidenceOfPlantAndAnimalCommun,3,2,2,0,1,0,1.0000,0.6667,0.8000,1.0000,0.6667,0.8000
5,source,8,8,5,3,3,6,0.6250,0.6250,0.6250,1.3750,1.3750,1.3750
6,fmicb-10-01018,83,35,2,33,81,59,0.0571,0.0241,0.0339,1.7429,0.7349,1.0339
7,Total,266,208,59,149,207,281,0.2837,0.2218,0.2490,1.6346,1.2782,1.4346


In [70]:
locationMetrics = getLocationsMetrics(LocationMatches)
locationMetrics

,Document,Tagged,Extracted,True Positives,False Positives,False Negatives,Partial Matches,Precision,Recall,F1,Adj. Precision,Adj. Recall,Adj. F1
0,Archer2017_Article_EndolithicMicrobialDiversityIn,96,33,12,21,84,40,0.3636,0.1250,0.1860,1.5758,0.5417,0.8062
1,fmicb-07-01642,87,32,21,11,66,22,0.6562,0.2414,0.3530,1.3438,0.4943,0.7227
2,summer-activity-patterns-for-mosses-and-lichen...,138,79,38,41,100,80,0.4810,0.2754,0.3503,1.4937,0.8551,1.0876
3,s42003-018-0260-y,35,9,5,4,30,8,0.5556,0.1429,0.2273,1.4444,0.3714,0.5909
4,Fraser2018_Article_EvidenceOfPlantAndAnimalCommun,64,35,18,17,46,32,0.5143,0.2812,0.3636,1.4286,0.7812,1.0101
5,source,31,9,8,1,23,2,0.8889,0.2581,0.4000,1.1111,0.3226,0.5000
6,fmicb-10-01018,95,53,18,35,77,68,0.3396,0.1895,0.2433,1.6226,0.9053,1.1622
7,Total,546,250,120,130,426,252,0.4800,0.2198,0.3015,1.4880,0.6813,0.9347


In [991]:
LocationMatches[
    LocationMatches.ExtractedLocation.isna()
#     LocationMatches.TaggedLocation.isna()
][['ExtractedLocation','Original Tokenised Text','Start_Token','End_Token','Sentence','TaggedLocation','Document']]

,ExtractedLocation,Original Tokenised Text,Start_Token,End_Token,Sentence,TaggedLocation,Document
33,NaN,NaN,NaN,NaN,NaN,Berlin,Archer2017_Article_EndolithicMicrobialDiversityIn
34,NaN,NaN,NaN,NaN,NaN,Heidelberg,Archer2017_Article_EndolithicMicrobialDiversityIn
35,NaN,NaN,NaN,NaN,NaN,McMurdo Dry Valleys,Archer2017_Article_EndolithicMicrobialDiversityIn
36,NaN,NaN,NaN,NaN,NaN,University Valley,Archer2017_Article_EndolithicMicrobialDiversityIn
37,NaN,NaN,NaN,NaN,NaN,Miers Valley,Archer2017_Article_EndolithicMicrobialDiversityIn
38,NaN,NaN,NaN,NaN,NaN,USA,Archer2017_Article_EndolithicMicrobialDiversityIn
39,NaN,NaN,NaN,NaN,NaN,Lubbock,Archer2017_Article_EndolithicMicrobialDiversityIn
40,NaN,NaN,NaN,NaN,NaN,TX,Archer2017_Article_EndolithicMicrobialDiversityIn
41,NaN,NaN,NaN,NaN,NaN,USA,Archer2017_Article_EndolithicMicrobialDiversityIn
42,NaN,NaN,NaN,NaN,NaN,University Valley,Archer2017_Article_EndolithicMicrobialDiversityIn


In [1014]:
# LocationMatches.loc[386]

In [992]:
LocationMatches[LocationMatches.ExtractedLocation.isna()]['TaggedLocation'].unique()

array(['Berlin', 'Heidelberg', 'McMurdo Dry Valleys', 'University Valley',
       'Miers Valley', 'USA', 'Lubbock', 'TX', 'Miers',
       'McKelvey valleys', 'McKelvey', 'Miers valleys', 'Miers Valleys',
       'Antarctic dry valleys', 'Tibetan plateau', 'Swedish Lapland',
       'Hawaii', 'Nanjiang', 'China', 'Spain', 'Antarctica', 'Auckland',
       'New Zealand', 'Princeton', 'NJ', 'Belfast', 'Northern Ireland',
       'Hong Kong', 'Madrid', 'Norman', 'OK', 'Beijing', 'Berkeley', 'CA',
       'Gloucestershire', 'UK', 'Kanazawa', 'Japan', 'Branford', 'CT',
       'NY', 'Offenburg', 'Germany', 'Basel', 'Switzerland', 'Kiel',
       'Hamilton', 'Salzburg', 'Austria', 'Livingston Island',
       'South Shetland Islands', 'Léonie Island', 'Taylor Valley',
       'Juan Carlos I Station', 'Mount Reina Sofia', 'Antarctic Circle',
       'Continental Antarctica', 'Livingston', 'Léonie islands',
       'Botany Bay', 'Garwood Valley', 'Canada Glacier', 'MDVs',
       'Antarctic continent', 'MD

## Create Matrix for Potential Relationships from Annotated Articles

In [1026]:
specAnn, locAnn, relAnn = getEntitiesandRelations('tagtog/BodyOnly','FraserSet/BodyOnly/Clean',1)

In [18]:
file = open('FraserSet/BodyOnly/Clean/Archer2017_Article_EndolithicMicrobialDiversityIn.txt')
document = file.read()
file.close()

In [19]:
doc = nlp_lg(document)

In [1029]:
specAnn

,Text,Start_Index,End_Index
96,Synechococcus,22640,22653
97,Microcoleus,22756,22767
98,Phormidium,22769,22779
99,Oscillatoria,22785,22797
157,Microcoleus,32042,32053
158,Oscillatoria,32055,32067
159,Synechococcus,32073,32086
162,Chroococcidiopsis,32671,32688
163,Microcoleus,32746,32757
168,Synechococcus,33513,33526


In [1033]:
SpeciesMatches.Document.unique()

array(['Archer2017_Article_EndolithicMicrobialDiversityIn',
       'fmicb-07-01642',
       'summer-activity-patterns-for-mosses-and-lichens-in-maritime-antarctica',
       's42003-018-0260-y',
       'Fraser2018_Article_EvidenceOfPlantAndAnimalCommun', 'source',
       'fmicb-10-01018'], dtype=object)

In [1071]:
SpeciesMatches[(SpeciesMatches.Document=='s42003-018-0260-y') & (SpeciesMatches.Found_as.isna())]

,Found_as,Full_name,Start_Token,End_Token,Start_Index,End_Index,Text,Match,Document
181,NaN,NaN,NaN,NaN,5721,5735,P. frigophilus,False,s42003-018-0260-y


In [1155]:
Test = SpeciesMatches[(SpeciesMatches.Document=='s42003-018-0260-y')]

In [1156]:
Test

,Found_as,Full_name,Start_Token,End_Token,Start_Index,End_Index,Text,Match,Document
113,Plectus murrayi,Plectus murrayi,927,929,5514,5529,NaN,False,s42003-018-0260-y
114,Eudorylaimus antarcticus,Eudorylaimus antarcticus,930,932,5531,5555,NaN,False,s42003-018-0260-y
115,Scottnema lindsayae,Scottnema lindsayae,934,936,5561,5580,NaN,False,s42003-018-0260-y
116,Geomonhystera antarcticola,Geomonhystera antarcticola,951,953,5654,5680,NaN,False,s42003-018-0260-y
117,Plectus,Plectus,960,961,5712,5719,NaN,False,s42003-018-0260-y
118,Scottnema,Scottnema,991,992,5898,5907,Scottnema,True,s42003-018-0260-y
119,Scottnema,Scottnema,1021,1022,6045,6054,NaN,False,s42003-018-0260-y
120,Plectus,Plectus,1023,1024,6056,6063,Plectus,True,s42003-018-0260-y
121,Eudorylaimus,Eudorylaimus,1029,1030,6093,6105,Eudorylaimus,True,s42003-018-0260-y
122,Plectus,Plectus,1055,1056,6237,6244,Plectus,True,s42003-018-0260-y


In [1115]:
doc[1].idx

10

In [1114]:
doc[1].sent == doc[2].sent

True

In [23]:
document[17566]

'L'

In [27]:
doc[3222:3223]

Leptolyngbya-

In [1170]:
document[21062:21071]

'Scottnema'

In [1181]:
doc[3595:3596].text

'Scottnema'

In [24]:
def get_token_num_for_char(doc, start_idx):
    for i, token in enumerate(doc):
        if start_idx > token.idx:
            continue
        if start_idx == token.idx:
            return i
        if start_idx < token.idx:
            return i - 1

In [28]:
def get_token_num_for_end_char(doc, start_token, text):
    end_token = start_token+1
    span=doc[start_token:end_token]
    print(span.text, start_token, end_token, text)
    while len(span.text)< len(text):
        end_token+=1
        span=doc[start_token:end_token]
        print('looping', span.text, start_token, end_token, text)
    return end_token

In [25]:
get_token_num_for_char(doc,17566)

3222

In [1179]:
get_token_num_for_end_char(doc, 7595)
#should be 1286

1285

In [1180]:
get_token_num_for_end_char(doc, 21071)
#should be 3596

3596

In [29]:
get_token_num_for_end_char(doc, 3222, 'Leptolyngbya')

Leptolyngbya- 3222 3223 Leptolyngbya


3223

In [1185]:
get_token_num_for_end_char(doc, 3595, 'Scottnema',21071)

3596

In [1226]:
get_token_num_for_end_char(doc, 2565, 'Hemichloris antarctica')

Hemichloris 2565 2566 Hemichloris antarctica
looping Hemichloris 2565 2567 Hemichloris antarctica
looping Hemichloris 2565 2568 Hemichloris antarctica
looping Hemichloris 2565 2569 Hemichloris antarctica
looping Hemichloris 2565 2570 Hemichloris antarctica
looping Hemichloris 2565 2571 Hemichloris antarctica
looping Hemichloris 2565 2572 Hemichloris antarctica
looping Hemichloris 2565 2573 Hemichloris antarctica
looping Hemichloris 2565 2574 Hemichloris antarctica
looping Hemichloris 2565 2575 Hemichloris antarctica
looping Hemichloris 2565 2576 Hemichloris antarctica
looping Hemichloris 2565 2577 Hemichloris antarctica
looping Hemichloris 2565 2578 Hemichloris antarctica
looping Hemichloris 2565 2579 Hemichloris antarctica
looping Hemichloris 2565 2580 Hemichloris antarctica
looping Hemichloris 2565 2581 Hemichloris antarctica
looping Hemichloris 2565 2582 Hemichloris antarctica
looping Hemichloris 2565 2583 Hemichloris antarctica
looping Hemichloris 2565 2584 Hemichloris antarctica
l

looping Hemichloris 2565 2946 Hemichloris antarctica
looping Hemichloris 2565 2947 Hemichloris antarctica
looping Hemichloris 2565 2948 Hemichloris antarctica
looping Hemichloris 2565 2949 Hemichloris antarctica
looping Hemichloris 2565 2950 Hemichloris antarctica
looping Hemichloris 2565 2951 Hemichloris antarctica
looping Hemichloris 2565 2952 Hemichloris antarctica
looping Hemichloris 2565 2953 Hemichloris antarctica
looping Hemichloris 2565 2954 Hemichloris antarctica
looping Hemichloris 2565 2955 Hemichloris antarctica
looping Hemichloris 2565 2956 Hemichloris antarctica
looping Hemichloris 2565 2957 Hemichloris antarctica
looping Hemichloris 2565 2958 Hemichloris antarctica
looping Hemichloris 2565 2959 Hemichloris antarctica
looping Hemichloris 2565 2960 Hemichloris antarctica
looping Hemichloris 2565 2961 Hemichloris antarctica
looping Hemichloris 2565 2962 Hemichloris antarctica
looping Hemichloris 2565 2963 Hemichloris antarctica
looping Hemichloris 2565 2964 Hemichloris anta

looping Hemichloris 2565 3246 Hemichloris antarctica
looping Hemichloris 2565 3247 Hemichloris antarctica
looping Hemichloris 2565 3248 Hemichloris antarctica
looping Hemichloris 2565 3249 Hemichloris antarctica
looping Hemichloris 2565 3250 Hemichloris antarctica
looping Hemichloris 2565 3251 Hemichloris antarctica
looping Hemichloris 2565 3252 Hemichloris antarctica
looping Hemichloris 2565 3253 Hemichloris antarctica
looping Hemichloris 2565 3254 Hemichloris antarctica
looping Hemichloris 2565 3255 Hemichloris antarctica
looping Hemichloris 2565 3256 Hemichloris antarctica
looping Hemichloris 2565 3257 Hemichloris antarctica
looping Hemichloris 2565 3258 Hemichloris antarctica
looping Hemichloris 2565 3259 Hemichloris antarctica
looping Hemichloris 2565 3260 Hemichloris antarctica
looping Hemichloris 2565 3261 Hemichloris antarctica
looping Hemichloris 2565 3262 Hemichloris antarctica
looping Hemichloris 2565 3263 Hemichloris antarctica
looping Hemichloris 2565 3264 Hemichloris anta

looping Hemichloris 2565 3582 Hemichloris antarctica
looping Hemichloris 2565 3583 Hemichloris antarctica
looping Hemichloris 2565 3584 Hemichloris antarctica
looping Hemichloris 2565 3585 Hemichloris antarctica
looping Hemichloris 2565 3586 Hemichloris antarctica
looping Hemichloris 2565 3587 Hemichloris antarctica
looping Hemichloris 2565 3588 Hemichloris antarctica
looping Hemichloris 2565 3589 Hemichloris antarctica
looping Hemichloris 2565 3590 Hemichloris antarctica
looping Hemichloris 2565 3591 Hemichloris antarctica
looping Hemichloris 2565 3592 Hemichloris antarctica
looping Hemichloris 2565 3593 Hemichloris antarctica
looping Hemichloris 2565 3594 Hemichloris antarctica
looping Hemichloris 2565 3595 Hemichloris antarctica
looping Hemichloris 2565 3596 Hemichloris antarctica
looping Hemichloris 2565 3597 Hemichloris antarctica
looping Hemichloris 2565 3598 Hemichloris antarctica
looping Hemichloris 2565 3599 Hemichloris antarctica
looping Hemichloris 2565 3600 Hemichloris anta

looping Hemichloris 2565 3946 Hemichloris antarctica
looping Hemichloris 2565 3947 Hemichloris antarctica
looping Hemichloris 2565 3948 Hemichloris antarctica
looping Hemichloris 2565 3949 Hemichloris antarctica
looping Hemichloris 2565 3950 Hemichloris antarctica
looping Hemichloris 2565 3951 Hemichloris antarctica
looping Hemichloris 2565 3952 Hemichloris antarctica
looping Hemichloris 2565 3953 Hemichloris antarctica
looping Hemichloris 2565 3954 Hemichloris antarctica
looping Hemichloris 2565 3955 Hemichloris antarctica
looping Hemichloris 2565 3956 Hemichloris antarctica
looping Hemichloris 2565 3957 Hemichloris antarctica
looping Hemichloris 2565 3958 Hemichloris antarctica
looping Hemichloris 2565 3959 Hemichloris antarctica
looping Hemichloris 2565 3960 Hemichloris antarctica
looping Hemichloris 2565 3961 Hemichloris antarctica
looping Hemichloris 2565 3962 Hemichloris antarctica
looping Hemichloris 2565 3963 Hemichloris antarctica
looping Hemichloris 2565 3964 Hemichloris anta

...

looping Hemichloris 2565 8245 Hemichloris antarctica
looping Hemichloris 2565 8246 Hemichloris antarctica
looping Hemichloris 2565 8247 Hemichloris antarctica
looping Hemichloris 2565 8248 Hemichloris antarctica
looping Hemichloris 2565 8249 Hemichloris antarctica
looping Hemichloris 2565 8250 Hemichloris antarctica
looping Hemichloris 2565 8251 Hemichloris antarctica
looping Hemichloris 2565 8252 Hemichloris antarctica
looping Hemichloris 2565 8253 Hemichloris antarctica
looping Hemichloris 2565 8254 Hemichloris antarctica
looping Hemichloris 2565 8255 Hemichloris antarctica
looping Hemichloris 2565 8256 Hemichloris antarctica
looping Hemichloris 2565 8257 Hemichloris antarctica
looping Hemichloris 2565 8258 Hemichloris antarctica
looping Hemichloris 2565 8259 Hemichloris antarctica
looping Hemichloris 2565 8260 Hemichloris antarctica
looping Hemichloris 2565 8261 Hemichloris antarctica
looping Hemichloris 2565 8262 Hemichloris antarctica
looping Hemichloris 2565 8263 Hemichloris anta

looping Hemichloris 2565 8545 Hemichloris antarctica
looping Hemichloris 2565 8546 Hemichloris antarctica
looping Hemichloris 2565 8547 Hemichloris antarctica
looping Hemichloris 2565 8548 Hemichloris antarctica
looping Hemichloris 2565 8549 Hemichloris antarctica
looping Hemichloris 2565 8550 Hemichloris antarctica
looping Hemichloris 2565 8551 Hemichloris antarctica
looping Hemichloris 2565 8552 Hemichloris antarctica
looping Hemichloris 2565 8553 Hemichloris antarctica
looping Hemichloris 2565 8554 Hemichloris antarctica
looping Hemichloris 2565 8555 Hemichloris antarctica
looping Hemichloris 2565 8556 Hemichloris antarctica
looping Hemichloris 2565 8557 Hemichloris antarctica
looping Hemichloris 2565 8558 Hemichloris antarctica
looping Hemichloris 2565 8559 Hemichloris antarctica
looping Hemichloris 2565 8560 Hemichloris antarctica
looping Hemichloris 2565 8561 Hemichloris antarctica
looping Hemichloris 2565 8562 Hemichloris antarctica
looping Hemichloris 2565 8563 Hemichloris anta

looping Hemichloris 2565 8845 Hemichloris antarctica
looping Hemichloris 2565 8846 Hemichloris antarctica
looping Hemichloris 2565 8847 Hemichloris antarctica
looping Hemichloris 2565 8848 Hemichloris antarctica
looping Hemichloris 2565 8849 Hemichloris antarctica
looping Hemichloris 2565 8850 Hemichloris antarctica
looping Hemichloris 2565 8851 Hemichloris antarctica
looping Hemichloris 2565 8852 Hemichloris antarctica
looping Hemichloris 2565 8853 Hemichloris antarctica
looping Hemichloris 2565 8854 Hemichloris antarctica
looping Hemichloris 2565 8855 Hemichloris antarctica
looping Hemichloris 2565 8856 Hemichloris antarctica
looping Hemichloris 2565 8857 Hemichloris antarctica
looping Hemichloris 2565 8858 Hemichloris antarctica
looping Hemichloris 2565 8859 Hemichloris antarctica
looping Hemichloris 2565 8860 Hemichloris antarctica
looping Hemichloris 2565 8861 Hemichloris antarctica
looping Hemichloris 2565 8862 Hemichloris antarctica
looping Hemichloris 2565 8863 Hemichloris anta

looping Hemichloris 2565 9156 Hemichloris antarctica
looping Hemichloris 2565 9157 Hemichloris antarctica
looping Hemichloris 2565 9158 Hemichloris antarctica
looping Hemichloris 2565 9159 Hemichloris antarctica
looping Hemichloris 2565 9160 Hemichloris antarctica
looping Hemichloris 2565 9161 Hemichloris antarctica
looping Hemichloris 2565 9162 Hemichloris antarctica
looping Hemichloris 2565 9163 Hemichloris antarctica
looping Hemichloris 2565 9164 Hemichloris antarctica
looping Hemichloris 2565 9165 Hemichloris antarctica
looping Hemichloris 2565 9166 Hemichloris antarctica
looping Hemichloris 2565 9167 Hemichloris antarctica
looping Hemichloris 2565 9168 Hemichloris antarctica
looping Hemichloris 2565 9169 Hemichloris antarctica
looping Hemichloris 2565 9170 Hemichloris antarctica
looping Hemichloris 2565 9171 Hemichloris antarctica
looping Hemichloris 2565 9172 Hemichloris antarctica
looping Hemichloris 2565 9173 Hemichloris antarctica
looping Hemichloris 2565 9174 Hemichloris anta

KeyboardInterrupt: 

In [1239]:
doc[9:13]

McMurdo Dry Valleys,

In [1126]:
document[5735]

','

In [1134]:
for row in SpeciesMatches[SpeciesMatches['Found_as'].isna()].index:
    SpeciesMatches.loc[row].Found_as = SpeciesMatches.loc[row].Text
    SpeciesMatches.loc[row].Start_Token = get_token_num_for_charSpeciesMatches.loc[row].Text

181


In [1203]:
del metrics

In [1212]:
del ActualResults

In [56]:
LocationMatches[LocationMatches.TaggedLocation.isna()]

,ExtractedLocation,Original Tokenised Text,Start_Token,End_Token,Start_Index,End_Index,Sentence,NZGazAnt,NZGaz,ScarNZ,...,PartialMatch_ScarNZ,PartialMatch_ScarGlobal,PartialMatch_GeoNamesNZ,PartialMatch_GeoNamesAnt,PartialMatch,inNZ,inAntarctica,Found,TaggedLocation,Document
12,Antarctic Dry Valley,Antarctic Dry Valley,1900,1903,10219,10239,Templates for positive and negative PCR contro...,False,False,False,...,NaN,NaN,NaN,NaN,True,True,True,True,NaN,Archer2017_Article_EndolithicMicrobialDiversityIn
118,McMurdo Dry Valleys of Antarctica,the McMurdo Dry Valleys of Antarctica,5101,5106,28263,28296,This study provides a combined interrogation o...,False,False,False,...,McMurdo Dry Valleys,McMurdo Dry Valleys,NaN,McMurdo Dry Valleys,True,True,True,True,NaN,fmicb-07-01642
221,Location,Location,1851,1852,9964,9972,Materials and methods Location and species,False,False,False,...,NaN,NaN,NaN,NaN,False,False,False,True,NaN,summer-activity-patterns-for-mosses-and-lichen...
324,McMurdo Dry Valleys (Antarctica,the McMurdo Dry Valleys (Antarctica,124,129,631,662,We modelled co-occurrence and distribution of ...,False,False,False,...,McMurdo Dry Valleys,McMurdo Dry Valleys,NaN,McMurdo Dry Valleys,True,True,True,True,NaN,s42003-018-0260-y
366,Lyon,Lyon,411,412,2382,2386,(Lyon and Giggenbach 1974).,False,False,False,...,NaN,NaN,NaN,NaN,False,False,False,True,NaN,Fraser2018_Article_EvidenceOfPlantAndAnimalCommun
383,Mount Rainier,Mt Rainier,1723,1724,9273,9283,"comm.), and are known from glaciated volcanoes...",False,False,False,...,NaN,NaN,NaN,NaN,False,False,False,True,NaN,Fraser2018_Article_EvidenceOfPlantAndAnimalCommun
496,West,West,8525,8526,44502,44506,"Thus, our findings suggest that a taxonomic re...",False,False,False,...,NaN,NaN,NaN,NaN,NaN,True,False,True,NaN,fmicb-10-01018


In [78]:
LocationMatches[LocationMatches.ExtractedLocation.isna()]

,ExtractedLocation,Original Tokenised Text,Start_Token,End_Token,Start_Index,End_Index,Sentence,NZGazAnt,NZGaz,ScarNZ,...,PartialMatch_ScarNZ,PartialMatch_ScarGlobal,PartialMatch_GeoNamesNZ,PartialMatch_GeoNamesAnt,PartialMatch,inNZ,inAntarctica,Found,TaggedLocation,Document
33,NaN,NaN,81,82,436,442,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Berlin,Archer2017_Article_EndolithicMicrobialDiversityIn
34,NaN,NaN,82,83,443,453,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Heidelberg,Archer2017_Article_EndolithicMicrobialDiversityIn
35,NaN,NaN,290,293,1831,1850,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,McMurdo Dry Valleys,Archer2017_Article_EndolithicMicrobialDiversityIn
36,NaN,NaN,1183,1185,6718,6735,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,University Valley,Archer2017_Article_EndolithicMicrobialDiversityIn
37,NaN,NaN,1217,1219,6936,6948,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miers Valley,Archer2017_Article_EndolithicMicrobialDiversityIn
38,NaN,NaN,1615,1616,9045,9048,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,USA,Archer2017_Article_EndolithicMicrobialDiversityIn
39,NaN,NaN,1964,1965,10564,10571,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lubbock,Archer2017_Article_EndolithicMicrobialDiversityIn
40,NaN,NaN,1966,1967,10573,10575,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TX,Archer2017_Article_EndolithicMicrobialDiversityIn
41,NaN,NaN,1968,1969,10577,10580,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,USA,Archer2017_Article_EndolithicMicrobialDiversityIn
42,NaN,NaN,2499,2501,13554,13571,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,University Valley,Archer2017_Article_EndolithicMicrobialDiversityIn


In [119]:
UnmatchedLocations = LocationMatches[LocationMatches.ExtractedLocation.isna()].copy()

In [103]:
LocationMatches.columns

Index(['ExtractedLocation', 'Original Tokenised Text', 'Start_Token',
       'End_Token', 'Start_Index', 'End_Index', 'Sentence', 'NZGazAnt',
       'NZGaz', 'ScarNZ', 'ScarGlobal', 'GeoNamesNZ', 'GeoNamesAnt',
       'GeoNames', 'exactMatch', 'Close_Match_NZGazAnt', 'Close_Match_NZGaz',
       'Close_Match_ScarNZ', 'Close_Match_ScarGlobal',
       'Close_Match_GeoNamesNZ', 'Close_Match_GeoNamesAnt', 'Close_Match',
       'PartialMatch_NZGazAnt', 'PartialMatch_NZGaz', 'PartialMatch_ScarNZ',
       'PartialMatch_ScarGlobal', 'PartialMatch_GeoNamesNZ',
       'PartialMatch_GeoNamesAnt', 'PartialMatch', 'inNZ', 'inAntarctica',
       'Found', 'TaggedLocation', 'Document'],
      dtype='object')

In [69]:
nzgaz = list(np.load('nzgazetteer.npy', allow_pickle=True))
nzgazAntartica = list(np.load('nzgazAntartica.npy', allow_pickle=True))
SCARGlobalnames = list(np.load('SCARGlobalnames.npy', allow_pickle=True))
SCARNZnames = list(np.load('SCARNZnames.npy', allow_pickle=True))
GeoNamesAnt = list(np.load('GeoNamesAnt.npy', allow_pickle=True))
GeoNamesNZ = list(np.load('GeoNamesNZ.npy', allow_pickle=True))
GeoNamesFiltered= list(np.load('GeoNamesFiltered.npy', allow_pickle=True))

In [177]:
LocationMatches.Close_Match_GeoNamesAnt.unique()

array([nan])

In [199]:
X = 'Mt Pines'
X[3:]

'Pines'

In [202]:
def populate_taggedlocations_df(df):
             
#     df.drop(columns=['NZGazAnt','NZGaz', 'ScarNZ', 'ScarGlobal', 'GeoNamesNZ', 'GeoNamesAnt',
#        'GeoNames', 'exactMatch', 'Close_Match_NZGazAnt', 'Close_Match_NZGaz',
#        'Close_Match_ScarNZ', 'Close_Match_ScarGlobal',
#        'Close_Match_GeoNamesNZ', 'Close_Match_GeoNamesAnt', 'Close_Match',
#        'PartialMatch_NZGazAnt', 'PartialMatch_NZGaz', 'PartialMatch_ScarNZ',
#        'PartialMatch_ScarGlobal', 'PartialMatch_GeoNamesNZ',
#        'PartialMatch_GeoNamesAnt', 'PartialMatch', 'inNZ', 'inAntarctica',
#        'Found'], inplace=True) 

    places=[]
    df['Location'] = [X[4:] if X[:3] == 'The' or X[:3] == 'the' 
                      else 'Mount ' + X[4:] if X[:3] == 'Mt.'
                      else 'Mount ' + X[3:] if X[:3] == 'Mt '
                      else X for X in 'TaggedLocations']
    
    
#     Check possible locations in gazetteers for exact matches
    df['NZGazAnt']=[True if X in nzgazAntartica else False for X in df.Location ]
    df['NZGaz']=[True if (df['NZGazAnt'] is True or X in nzgaz) else False for X in df.Location ]
    df['ScarNZ']=[True if X in SCARNZnames else False for X in df.Location ]
    df['ScarGlobal']=[True if X in SCARGlobalnames else False for X in df.Location]
    df['GeoNamesNZ']=[True if X in GeoNamesNZ else False for X in df.Location]
    df['GeoNamesAnt']=[True if X in GeoNamesAnt else False for X in df.Location]
    df['GeoNames']=[True if (df['GeoNamesNZ'] is True or df['GeoNamesAnt'] is True or X in GeoNamesFiltered) else False for X in df.Location ]
    df['inNZ'] = df.NZGaz | df.NZGazAnt | df.GeoNamesNZ
    df['inAntarctica'] = df.ScarGlobal | df.NZGazAnt | df.GeoNamesAnt
    df['exactMatch'] = df.NZGaz | df.ScarGlobal | df.GeoNames
    
    
#     filter locations by those not found in Antarctica or New Zealand gazeteers 
    df2=df[(df['inAntarctica']==False) & (df['inNZ']==False)].copy()
    

#     look through these remaining locations (including those found only in GeoNames) for close matches 
#     eg. McMurdo Dry Valley v McMurdo Dry Valley or for partial matches eg. Ross Sea Region == Ross Sea 
   
    df2['Close_Match_NZGazAnt']=  [difflib.get_close_matches(X, nzgazAntartica, cutoff=0.9)[0]
                                                  if len(difflib.get_close_matches(X, nzgazAntartica, cutoff=0.9))>0
                                                  else np.nan 
                                                  for X in df2['Location']] 
    df2['Close_Match_NZGaz']=  [difflib.get_close_matches(X, nzgaz, cutoff=0.9)[0]
                                                      if len(difflib.get_close_matches(X, nzgaz, cutoff=0.9))>0
                                                      else np.nan 
                                                      for X in df2['Location']] 
    df2['Close_Match_ScarNZ']=  [difflib.get_close_matches(X, SCARNZnames, cutoff=0.9)[0] 
                                                      if len(difflib.get_close_matches(X, SCARNZnames, cutoff=0.9))>0
                                                      else np.nan 
                                                      for X in df2['Location']] 
    df2['Close_Match_ScarGlobal']=  [difflib.get_close_matches(X, SCARGlobalnames, cutoff=0.9)[0]
                                                      if (X !='Antarctic' and len(difflib.get_close_matches(X, SCARGlobalnames, cutoff=0.9))>0)
                                                      else np.nan 
                                                      for X in df2['Location']] 
    df2['Close_Match_GeoNamesNZ'] = [difflib.get_close_matches(X, GeoNamesNZ, cutoff=0.9)[0] 
                                                      if len(difflib.get_close_matches(X, GeoNamesNZ, cutoff=0.9))>0
                                                      else np.nan 
                                                      for X in df2['Location']] 
    df2['Close_Match_GeoNamesAnt']=  [difflib.get_close_matches(X, GeoNamesAnt, cutoff=0.9)[0]
                                                      if (X !='Antarctic' and len(difflib.get_close_matches(X, GeoNamesAnt, cutoff=0.9))>0)
                                                      else np.nan 
                                                      for X in df2['Location']]
    df2['Close_Match']= (df2.Close_Match_NZGazAnt.notna() | df2.Close_Match_NZGaz.notna() | 
                         df2.Close_Match_ScarNZ.notna() | df2.Close_Match_ScarGlobal.notna() | 
                         df2.Close_Match_GeoNamesAnt.notna())
    
    df2['PartialMatch_NZGazAnt']=getBiggestSubStringMatch(doc, df2, nzgazAntartica)
    df2['PartialMatch_NZGaz']=getBiggestSubStringMatch(doc, df2, nzgaz)
    df2['PartialMatch_ScarNZ']=getBiggestSubStringMatch(doc, df2, SCARNZnames)
    df2['PartialMatch_ScarGlobal']=getBiggestSubStringMatch(doc, df2, SCARGlobalnames)
    df2['PartialMatch_GeoNamesNZ']=getBiggestSubStringMatch(doc, df2, GeoNamesNZ)
    df2['PartialMatch_GeoNamesAnt']=getBiggestSubStringMatch(doc, df2, GeoNamesAnt)
    df2['PartialMatch'] = (df2['PartialMatch_NZGazAnt'].notna() | df2['PartialMatch_NZGaz'].notna() | 
                           df2['PartialMatch_ScarNZ'].notna() | df2['PartialMatch_ScarGlobal'].notna() | 
                           df2['PartialMatch_GeoNamesAnt'].notna())
#     df2['PartialMatch'] = [X if X==True else np.nan for X in df2['PartialMatch']]
    
#     merge the filtered dataframe (now with close and partial matches) with the unfiltered dataframe
    df_unified = pd.merge(df, df2, how='left')
    
#     redo the inNZ,inAntarctica and Found columns to include close and partial matches
    df_unified.drop(columns=['inNZ', 'inAntarctica'], inplace=True)
    df_unified['inNZ'] = (df_unified.NZGaz | df_unified.NZGazAnt | df_unified.GeoNamesNZ | 
                          df_unified.Close_Match_NZGaz | df_unified.Close_Match_NZGazAnt | 
                          df_unified.Close_Match_GeoNamesNZ | df_unified.PartialMatch_NZGaz | 
                          df_unified.PartialMatch_NZGazAnt | df_unified.PartialMatch_GeoNamesNZ)
    df_unified['inAntarctica'] = (df_unified.ScarGlobal | df_unified.NZGazAnt | df_unified.GeoNamesAnt |
                                  df_unified.Close_Match_ScarGlobal | df_unified.Close_Match_NZGazAnt | 
                                  df_unified.Close_Match_GeoNamesAnt | df_unified.PartialMatch_ScarGlobal | 
                                  df_unified.PartialMatch_NZGazAnt | df_unified.PartialMatch_GeoNamesAnt)
    df_unified['Found'] = df_unified.exactMatch | df_unified.Close_Match | df_unified.PartialMatch
#     return df_unified[df_unified.Found==True]
    return df_unified

In [202]:
def populate_taggedlocations_df(df):
             
#     df.drop(columns=['NZGazAnt','NZGaz', 'ScarNZ', 'ScarGlobal', 'GeoNamesNZ', 'GeoNamesAnt',
#        'GeoNames', 'exactMatch', 'Close_Match_NZGazAnt', 'Close_Match_NZGaz',
#        'Close_Match_ScarNZ', 'Close_Match_ScarGlobal',
#        'Close_Match_GeoNamesNZ', 'Close_Match_GeoNamesAnt', 'Close_Match',
#        'PartialMatch_NZGazAnt', 'PartialMatch_NZGaz', 'PartialMatch_ScarNZ',
#        'PartialMatch_ScarGlobal', 'PartialMatch_GeoNamesNZ',
#        'PartialMatch_GeoNamesAnt', 'PartialMatch', 'inNZ', 'inAntarctica',
#        'Found'], inplace=True) 

    places=[]
    df['Location'] = [X[4:] if X[:3] == 'The' or X[:3] == 'the' 
                      else 'Mount ' + X[4:] if X[:3] == 'Mt.'
                      else 'Mount ' + X[3:] if X[:3] == 'Mt '
                      else X for X in 'TaggedLocations']
    
    
#     Check possible locations in gazetteers for exact matches
    df['NZGazAnt']=[True if X in nzgazAntartica else False for X in df.Location ]
    df['NZGaz']=[True if (df['NZGazAnt'] is True or X in nzgaz) else False for X in df.Location ]
    df['ScarNZ']=[True if X in SCARNZnames else False for X in df.Location ]
    df['ScarGlobal']=[True if X in SCARGlobalnames else False for X in df.Location]
    df['GeoNamesNZ']=[True if X in GeoNamesNZ else False for X in df.Location]
    df['GeoNamesAnt']=[True if X in GeoNamesAnt else False for X in df.Location]
    df['GeoNames']=[True if (df['GeoNamesNZ'] is True or df['GeoNamesAnt'] is True or X in GeoNamesFiltered) else False for X in df.Location ]
    df['inNZ'] = df.NZGaz | df.NZGazAnt | df.GeoNamesNZ
    df['inAntarctica'] = df.ScarGlobal | df.NZGazAnt | df.GeoNamesAnt
    df['exactMatch'] = df.NZGaz | df.ScarGlobal | df.GeoNames
    
    
#     filter locations by those not found in Antarctica or New Zealand gazeteers 
    df2=df[(df['inAntarctica']==False) & (df['inNZ']==False)].copy()
    

#     look through these remaining locations (including those found only in GeoNames) for close matches 
#     eg. McMurdo Dry Valley v McMurdo Dry Valley or for partial matches eg. Ross Sea Region == Ross Sea

    for row in df[(df['inAntarctica']==False) & (df['inNZ']==False)].index:
        if len(difflib.get_close_matches(X, nzgazAntartica, cutoff=0.9))>0:
            df.loc[row]['Close_Match_NZGazAnt']= difflib.get_close_matches(df2.loc[row]['Location'], 
                                                                           nzgazAntartica, cutoff=0.9)[0]
        if len(difflib.get_close_matches(X, nzgaz, cutoff=0.9))>0:
            df.loc[row]['Close_Match_NZGaz']= difflib.get_close_matches(df2.loc[row]['Location'], 
                                                                           nzgaz, cutoff=0.9)[0]    
        if len(difflib.get_close_matches(X, SCARNZnames, cutoff=0.9))>0:
            df.loc[row]['Close_Match_ScarNZ']= difflib.get_close_matches(df2.loc[row]['Location'], 
                                                                           SCARNZnames, cutoff=0.9)[0]       


    df2['Close_Match_ScarNZ']=  [difflib.get_close_matches(X, SCARNZnames, cutoff=0.9)[0] 
                                                      if len(difflib.get_close_matches(X, SCARNZnames, cutoff=0.9))>0
                                                      else np.nan 
                                                      for X in df2['Location']] 
    df2['Close_Match_ScarGlobal']=  [difflib.get_close_matches(X, SCARGlobalnames, cutoff=0.9)[0]
                                                      if (X !='Antarctic' and len(difflib.get_close_matches(X, SCARGlobalnames, cutoff=0.9))>0)
                                                      else np.nan 
                                                      for X in df2['Location']] 
    df2['Close_Match_GeoNamesNZ'] = [difflib.get_close_matches(X, GeoNamesNZ, cutoff=0.9)[0] 
                                                      if len(difflib.get_close_matches(X, GeoNamesNZ, cutoff=0.9))>0
                                                      else np.nan 
                                                      for X in df2['Location']] 
    df2['Close_Match_GeoNamesAnt']=  [difflib.get_close_matches(X, GeoNamesAnt, cutoff=0.9)[0]
                                                      if (X !='Antarctic' and len(difflib.get_close_matches(X, GeoNamesAnt, cutoff=0.9))>0)
                                                      else np.nan 
                                                      for X in df2['Location']]
    df2['Close_Match']= (df2.Close_Match_NZGazAnt.notna() | df2.Close_Match_NZGaz.notna() | 
                         df2.Close_Match_ScarNZ.notna() | df2.Close_Match_ScarGlobal.notna() | 
                         df2.Close_Match_GeoNamesAnt.notna())
    
    df2['PartialMatch_NZGazAnt']=getBiggestSubStringMatch(doc, df2, nzgazAntartica)
    df2['PartialMatch_NZGaz']=getBiggestSubStringMatch(doc, df2, nzgaz)
    df2['PartialMatch_ScarNZ']=getBiggestSubStringMatch(doc, df2, SCARNZnames)
    df2['PartialMatch_ScarGlobal']=getBiggestSubStringMatch(doc, df2, SCARGlobalnames)
    df2['PartialMatch_GeoNamesNZ']=getBiggestSubStringMatch(doc, df2, GeoNamesNZ)
    df2['PartialMatch_GeoNamesAnt']=getBiggestSubStringMatch(doc, df2, GeoNamesAnt)
    df2['PartialMatch'] = (df2['PartialMatch_NZGazAnt'].notna() | df2['PartialMatch_NZGaz'].notna() | 
                           df2['PartialMatch_ScarNZ'].notna() | df2['PartialMatch_ScarGlobal'].notna() | 
                           df2['PartialMatch_GeoNamesAnt'].notna())
#     df2['PartialMatch'] = [X if X==True else np.nan for X in df2['PartialMatch']]
    
#     merge the filtered dataframe (now with close and partial matches) with the unfiltered dataframe
    df_unified = pd.merge(df, df2, how='left')
    
#     redo the inNZ,inAntarctica and Found columns to include close and partial matches
    df_unified.drop(columns=['inNZ', 'inAntarctica'], inplace=True)
    df_unified['inNZ'] = (df_unified.NZGaz | df_unified.NZGazAnt | df_unified.GeoNamesNZ | 
                          df_unified.Close_Match_NZGaz | df_unified.Close_Match_NZGazAnt | 
                          df_unified.Close_Match_GeoNamesNZ | df_unified.PartialMatch_NZGaz | 
                          df_unified.PartialMatch_NZGazAnt | df_unified.PartialMatch_GeoNamesNZ)
    df_unified['inAntarctica'] = (df_unified.ScarGlobal | df_unified.NZGazAnt | df_unified.GeoNamesAnt |
                                  df_unified.Close_Match_ScarGlobal | df_unified.Close_Match_NZGazAnt | 
                                  df_unified.Close_Match_GeoNamesAnt | df_unified.PartialMatch_ScarGlobal | 
                                  df_unified.PartialMatch_NZGazAnt | df_unified.PartialMatch_GeoNamesAnt)
    df_unified['Found'] = df_unified.exactMatch | df_unified.Close_Match | df_unified.PartialMatch
#     return df_unified[df_unified.Found==True]
    return df_unified

In [76]:
def getBiggestSubStringMatch(document, df, gazetteer):
    matches=[]
    for row in df.index:
#         print('working on row', row)
        fullStart = df.loc[row,'Start_Token']
        fullEnd = df.loc[row,'End_Token']
        subLength = fullEnd-fullStart-1
#         print(fullEnd, fullStart, subLength)
        Found = False
        while subLength >1:
            subStart = fullStart
            subEnd = fullStart+subLength
            while subEnd<=fullEnd:
                subString = document[subStart:subEnd].text
#                 print('looking for', subString)
                if subString in gazetteer:
                    matches.append(subString)
                    Found=True,
#                     print('found it!')
                    break
                subStart+=1
                subEnd+=1
            if Found:
                    break
            else:
                subLength-=1
        if Found==False:
            matches.append(np.nan)
    return matches

In [120]:
UnmatchedLocations = populate_taggedlocations_df(UnmatchedLocations)

In [113]:
UnmatchedLocations.head()

,ExtractedLocation,Original Tokenised Text,Start_Token,End_Token,Start_Index,End_Index,Sentence,TaggedLocation,Document,NZGazAnt,...,PartialMatch_NZGazAnt,PartialMatch_NZGaz,PartialMatch_ScarNZ,PartialMatch_ScarGlobal,PartialMatch_GeoNamesNZ,PartialMatch_GeoNamesAnt,PartialMatch,inNZ,inAntarctica,Found
0,NaN,NaN,81,82,436,442,NaN,Berlin,Archer2017_Article_EndolithicMicrobialDiversityIn,False,...,NaN,NaN,NaN,NaN,NaN,NaN,False,True,False,True
1,NaN,NaN,82,83,443,453,NaN,Heidelberg,Archer2017_Article_EndolithicMicrobialDiversityIn,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,True
2,NaN,NaN,290,293,1831,1850,NaN,McMurdo Dry Valleys,Archer2017_Article_EndolithicMicrobialDiversityIn,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True,True
3,NaN,NaN,1183,1185,6718,6735,NaN,University Valley,Archer2017_Article_EndolithicMicrobialDiversityIn,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True,True
4,NaN,NaN,1217,1219,6936,6948,NaN,Miers Valley,Archer2017_Article_EndolithicMicrobialDiversityIn,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True,True


In [130]:
LocationMatches.update(UnmatchedLocations, overwrite=True)

In [132]:
LocationMatches

,ExtractedLocation,Original Tokenised Text,Start_Token,End_Token,Start_Index,End_Index,Sentence,NZGazAnt,NZGaz,ScarNZ,...,PartialMatch_ScarNZ,PartialMatch_ScarGlobal,PartialMatch_GeoNamesNZ,PartialMatch_GeoNamesAnt,PartialMatch,inNZ,inAntarctica,Found,TaggedLocation,Document
0,McMurdo Dry Valleys,the McMurdo Dry Valleys,81.0,82.0,436.0,442.0,Endolithic microbial diversity in sandstone an...,False,False,False,...,NaN,NaN,NaN,NaN,False,True,False,True,Berlin,Archer2017_Article_EndolithicMicrobialDiversityIn
1,Antarctica,Antarctica,82.0,83.0,443.0,453.0,"Antarctica Stephen D. J. Archer1,2 •",False,True,False,...,NaN,NaN,NaN,NaN,NaN,True,False,True,Heidelberg,Archer2017_Article_EndolithicMicrobialDiversityIn
2,Antarctica,Antarctica,290.0,293.0,1831.0,1850.0,Abstract Cryptic microbial communities develop...,True,True,True,...,NaN,NaN,NaN,NaN,NaN,True,True,True,McMurdo Dry Valleys,Archer2017_Article_EndolithicMicrobialDiversityIn
3,McMurdo Dry Valleys,McMurdo Dry Valleys,1183.0,1185.0,6718.0,6735.0,Abstract Cryptic microbial communities develop...,True,True,True,...,NaN,NaN,NaN,NaN,NaN,True,True,True,University Valley,Archer2017_Article_EndolithicMicrobialDiversityIn
4,Mars,Mars,1217.0,1219.0,6936.0,6948.0,The study demonstrates that endolithic microbi...,True,True,True,...,NaN,NaN,NaN,NaN,False,True,True,True,Miers Valley,Archer2017_Article_EndolithicMicrobialDiversityIn
5,Antarctica,Antarctica,1615.0,1616.0,9045.0,9048.0,Keywords Antarctica 􏰀,False,False,False,...,NaN,NaN,NaN,NaN,False,False,False,False,USA,Archer2017_Article_EndolithicMicrobialDiversityIn
6,Antarctica,Antarctica,1964.0,1965.0,10564.0,10571.0,The McMurdo Dry Valleys are the largest ice-fr...,False,False,False,...,NaN,NaN,NaN,NaN,False,False,False,True,Lubbock,Archer2017_Article_EndolithicMicrobialDiversityIn
7,Earth,Earth,1966.0,1967.0,10573.0,10575.0,The McMurdo Dry Valleys are the largest ice-fr...,False,False,False,...,NaN,NaN,NaN,NaN,False,False,False,False,TX,Archer2017_Article_EndolithicMicrobialDiversityIn
8,McMurdo Dry Valleys,the McMurdo Dry Valleys,1968.0,1969.0,10577.0,10580.0,Here we report a study to identify the taxonom...,False,False,False,...,NaN,NaN,NaN,NaN,False,False,False,False,USA,Archer2017_Article_EndolithicMicrobialDiversityIn
9,Antarctica,Antarctica,2499.0,2501.0,13554.0,13571.0,Here we report a study to identify the taxonom...,True,True,True,...,NaN,NaN,NaN,NaN,NaN,True,True,True,University Valley,Archer2017_Article_EndolithicMicrobialDiversityIn


In [123]:
for col in asBool:
    UnmatchedLocations[col] = UnmatchedLocations[col].astype(object)
    LocationMatches[col] = LocationMatches[col].astype(object)

In [117]:
UnmatchedLocations.PartialMatch.unique()

array([False,  True])

In [118]:
LocationMatches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 553 entries, 0 to 552
Data columns (total 34 columns):
ExtractedLocation           250 non-null object
Original Tokenised Text     250 non-null object
Start_Token                 553 non-null int64
End_Token                   553 non-null int64
Start_Index                 553 non-null int64
End_Index                   553 non-null int64
Sentence                    250 non-null object
NZGazAnt                    250 non-null object
NZGaz                       250 non-null object
ScarNZ                      250 non-null object
ScarGlobal                  250 non-null object
GeoNamesNZ                  250 non-null object
GeoNamesAnt                 250 non-null object
GeoNames                    250 non-null object
exactMatch                  250 non-null object
Close_Match_NZGazAnt        59 non-null object
Close_Match_NZGaz           59 non-null object
Close_Match_ScarNZ          3 non-null object
Close_Match_ScarGlobal      4 non-null 

In [134]:
LocationMatches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 553 entries, 0 to 552
Data columns (total 34 columns):
ExtractedLocation           250 non-null object
Original Tokenised Text     250 non-null object
Start_Token                 553 non-null float64
End_Token                   553 non-null float64
Start_Index                 553 non-null float64
End_Index                   553 non-null float64
Sentence                    250 non-null object
NZGazAnt                    409 non-null object
NZGaz                       409 non-null object
ScarNZ                      409 non-null object
ScarGlobal                  409 non-null object
GeoNamesNZ                  409 non-null object
GeoNamesAnt                 409 non-null object
GeoNames                    409 non-null object
exactMatch                  409 non-null object
Close_Match_NZGazAnt        66 non-null object
Close_Match_NZGaz           71 non-null object
Close_Match_ScarNZ          9 non-null object
Close_Match_ScarGlobal      14 

In [137]:
LocationMatches[LocationMatches.ExtractedLocation.isna()].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 303 entries, 33 to 552
Data columns (total 34 columns):
ExtractedLocation           0 non-null object
Original Tokenised Text     0 non-null object
Start_Token                 303 non-null float64
End_Token                   303 non-null float64
Start_Index                 303 non-null float64
End_Index                   303 non-null float64
Sentence                    0 non-null object
NZGazAnt                    159 non-null object
NZGaz                       159 non-null object
ScarNZ                      159 non-null object
ScarGlobal                  159 non-null object
GeoNamesNZ                  159 non-null object
GeoNamesAnt                 159 non-null object
GeoNames                    159 non-null object
exactMatch                  159 non-null object
Close_Match_NZGazAnt        3 non-null object
Close_Match_NZGaz           6 non-null object
Close_Match_ScarNZ          2 non-null object
Close_Match_ScarGlobal      5 non-null

In [170]:
UnmatchedLocations[UnmatchedLocations.Found.isna()]

,ExtractedLocation,Original Tokenised Text,Start_Token,End_Token,Start_Index,End_Index,Sentence,TaggedLocation,Document,NZGazAnt,...,PartialMatch_NZGazAnt,PartialMatch_NZGaz,PartialMatch_ScarNZ,PartialMatch_ScarGlobal,PartialMatch_GeoNamesNZ,PartialMatch_GeoNamesAnt,PartialMatch,inNZ,inAntarctica,Found


In [166]:
TaggedLocations = LocationMatches[LocationMatches.TaggedLocation.notna()]

In [168]:
TaggedLocations.tail()

,ExtractedLocation,Original Tokenised Text,Start_Token,End_Token,Start_Index,End_Index,Sentence,NZGazAnt,NZGaz,ScarNZ,...,PartialMatch_ScarNZ,PartialMatch_ScarGlobal,PartialMatch_GeoNamesNZ,PartialMatch_GeoNamesAnt,PartialMatch,inNZ,inAntarctica,Found,TaggedLocation,Document
548,NaN,NaN,7680.0,7682.0,39842.0,39857.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Victoria Valley,fmicb-10-01018
549,NaN,NaN,7984.0,7986.0,41511.0,41526.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Galindez Island,fmicb-10-01018
550,NaN,NaN,7988.0,7989.0,41537.0,41547.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Antarctica,fmicb-10-01018
551,NaN,NaN,8414.0,8417.0,43904.0,43927.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,eastern Antarctic lakes,fmicb-10-01018
552,NaN,NaN,9810.0,9813.0,51594.0,51616.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Antarctica Dry Valleys,fmicb-10-01018
